<a href="https://colab.research.google.com/github/shintaroudlotulhanafia/DRLforMultipleStocksTradingUsingFinRL/blob/main/StockFractionRobustness/64_Fraksi3_TugasAkhirS1_Shinta_DRLforMultipleStocksTradingUsingFinRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data saham-saham fraksi 3 dijalankan menggunakan email shintarh18@gmail.com

# Deep Reinforcement Learning untuk Jual-Beli Saham (Dari Awal): Jual-beli Banyak Saham 

* **Pytorch Version** 



**Memasangkan Google Drive ke Google Colab**

* Kata kunci **from** digunakan untuk mengimpor hanya bagian tertentu dari modul (melakukan impor untuk bagian tertentu pada *library*)
* Modul merupakan file yang berisi sekumpulan fungsi yang ingin disertakan dalam aplikasi. Untuk membuat modul cukup simpan kode yang Anda inginkan dalam file dengan ekstensi file .py.
* **import** melakukan impor seluruh *library*.
* **Mounting** adalah proses membuat file dan direktori pada perangkat penyimpanan (seperti google drive) sehingga tersedia bagi pengguna untuk dapat diakses.



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Daftar Isi:

* [1. Pendeskrisian Tugas](#0)
* [2. Instalasi Paket Python](#1)
    * [2.1. Instalasi Paket](#1.1)    
    * [2.2. Daftar Paket Python](#1.2)
    * [2.3. Impot Paket](#1.3)
    * [2.4. Membuat Folder](#1.4)
* [3. Mengunduh Data](#2)
* [4. Memproses Data](#3)        
    * [4.1. IndiKator Teknikal](#3.1)
    * [4.2. Melakukan *Feature Engineering*](#3.2)
* [5. Membangun Lingkungan Jual-Beli Saham dengan OpenAI Gym-style](#4)  
    * [5.1. Pemisahan Data](#4.1)  
    * [5.2. Lingkungan untuk Pelatihan *(Training)*](#4.2)    
* [6. Melatih Agen DRL](#5)
    * [6.1. Agen 1: A2C](#5.1)
    * [6.2. Agen 2: PPO](#5.2)
    * [6.3. Agen 3: TD3](#5.3)
    * [6.4. Agen 4: SAC](#5.4)
    * [6.5. Agen 5: DDPG](#5.5)
* [7. Melakukan Jual-Beli](#6)
    * [7.1. Performa di Dalam Sampel](#6.1)
    * [7.2. Performa di Luar Sampel](#6.2)
    * [7.3. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi](#6.3)
      * [7.3.1. Agen 1: A2C](#6.3.1)
      * [7.3.2. Agen 2: PPO](#6.3.2)
      * [7.3.3. Agen 3: TD3](#6.3.3)
      * [7.3.4. Agen 4: SAC](#6.3.4)
      * [7.3.5. Agen 5: DDPG](#6.3.5)
    * [7.4. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi (dalam bentuk string)](#6.3)
      * [7.4.1. Agen 1: A2C](#6.4.1)
      * [7.4.2. Agen 2: PPO](#6.4.2)
      * [7.4.3. Agen 3: TD3](#6.4.3)
      * [7.4.4. Agen 4: SAC](#6.4.4)
      * [7.4.5. Agen 5: DDPG](#6.4.5)
    * [7.5. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Kondisi dan Aksi](#6.4)
      * [7.5.1. Agen 1: A2C](#6.4.1)
      * [7.5.2. Agen 2: PPO](#6.4.2)
      * [7.5.3. Agen 3: TD3](#6.4.3)
      * [7.5.4. Agen 4: SAC](#6.4.4)
      * [7.5.5. Agen 5: DDPG](#6.4.5)
* [8. Performa Backtesting](#7)  
    * [8.1. Status BackTesting](#7.1)

<a id='0'></a>
# Bagian 1. Pendeskrisian Tugas

Agen DRL dilatih untuk melakukan jual-beli saham. Tugas tersebut dimodelkan sebagai Markov Decision Process (MDP), dengan fungsi dan tujuannya adalah memaksimalkan pengembalian pengembalian kumulatif *(cummulative return)* yang diharapkan.

Definisi state-action-reward pada algoritam DRL kasus kali ini adalah sebagai berikut:

* **State s**: Kondisi atau *state* mewakili persepsi agen tentang lingkungan pasar. Sama seperti *trader* manusia yang menganalisis berbagai informasi, agen juga secara pasif mengamati banyak fitur dan belajar dengan cara berinteraksi dengan lingkungan pasar (biasanya dengan menjalankan ulang data historis).

* **Tindakan atau *action* a**: Ruang aksi mencakup aksi atau tindakan yang dapat dilakukan agen di setiap status atau kondisi. Misalnya, a {−1, 0, 1}, -1 berarti menjual, 0 berarti menahan, dan 1 berarti membeli. Ketika suatu aksi mengoperasikan beberapa saham, maka, a {−k, ..., 1, 0, 1, ..., k}. Misalnya, "Beli
10 saham TLKM" atau "Jual 10 saham TLKM" maka masing-masing nilai a adalah 10 atau -10.

* **Fungsi *reward* atau imbalan r(s, a, s′)**: *Reward* adalah insentif bagi agen untuk mempelajari kebijakan yang lebih baik. Misalnya *reward* dapat berupa perubahan nilai portofolio saat mengambil a pada keadaan s dan tiba pada keadaan baru s', yaitu, r(s, a, s′) = v′-v, v′ mewakili nilai portofolio pada keadaan s′ dan v mewakili nilai portofolio pada keadaan s.

* **Environment atau lingkungan jual-beli**: saham penyusun indeks JII dengan tanggal sesuai periode pengujian yang diatur.


Data untuk studi kasus ini diperoleh dari Yahoo Finance API. Data berisi harga *Open-High-Low-Close* dan *Volume*.

<a id='1'></a>
# Bagian 2. Instalasi Paket Python

<a id='1.1'></a>
## 2.1. Instalasi Paket


* FinRL adalah *open-source framework* pertama yang menunjukkan potensi besar *Reinforcement Learning* keuangan.

* FinRL menyediakan berbagai pengaturan untuk melakukan *trading* dengan *Reinforcement Learning* seperti menyediakan ratusan pasar keuangan, algoritma yang canggih, berbagai macam aplikasi keuangan (alokasi portofolio, perdagangan mata uang kripto, *high-frequency trading*), *live trading, cloud deployment,* dll.

* Pengaplikasian pustaka FinRL dapat menggunakan alamat berikut, git+https://github.com/AI4Finance-Foundation/FinRL.git. Namun, untuk kemudahan pengaturan nilai sesuai kebutuhan, maka, dilakukanlah *forking* terhadap *repository* tersebut, lalu digunakanlah alamat berikut, git+https://github.com/shintaroudlotulhanafia/FinRL.git



In [2]:
# install finrl library
!pip install git+https://github.com/shintaroudlotulhanafia/FinRL.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/shintaroudlotulhanafia/FinRL.git to /tmp/pip-req-build-ospro2pr
  Running command git clone -q https://github.com/shintaroudlotulhanafia/FinRL.git /tmp/pip-req-build-ospro2pr
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-8_kujn9t/pyfolio_ba29ec703e2b4eab9cd747ca102d9de3
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-8_kujn9t/pyfolio_ba29ec703e2b4eab9cd747ca102d9de3
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-8_kujn9t/elegantrl_87267c129b4345dda39c8a5ce1dff320
  Running command git clone -q https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-8_kujn9t/elegantrl_87267c129b4345dda39c8a5ce1dff320
     |████████████████████████████████| 2.3 MB 11.3 MB/s 
     |████████████████████████████████| 234 kB 85.9 MB/s 
     |█████████████████████

<a id='1.2'></a>
## 2.2. Daftar Paket Python
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Impot Paket

* Modul Python **pandas** digunakan untuk menganalisis dan memanipulasi data.
* Modul Python **numpy** digunakan untuk memproses larik atau array.
* Modul Python **matplotlib** digunakan membuat visualisasi data dalam dua dimensi.
* Modul Python **matplotlib.pyplot** adalah kumpulan fungsi yang membuat matplotlib berfungsi seperti MATLAB.
* Modul Python **Datetime** menyediakan sejumlah fungsi untuk menangani tanggal, waktu, dan interval waktu. Date dan datetime adalah objek dalam Python, bukan string atau timestamps.
* **YahooDownloader** menyediakan metode untuk mengambil data saham harian dari API Keuangan Yahoo!
* **FeatureEngineer** menyediakan metode untuk preprocessing data harga saham
* **data_split** membagi dataset menjadi data pelatihan dan data pengujian berdasarkan tanggal
* **StockTradingEnv** Lingkungan perdagangan saham untuk OpenAI gym
* **DRLAgent** menyediakan implementasi untuk algoritma DRL
* **DataProcessor** memproses data menggunakan prosesor data terpadu
* **backtest_stats** menghitung statistik *backtesting*
* **backtest_plot** membuat dan menampilkan plot ringkasan laporan *backtesting*.
* **get_daily_return** 
* **get_baseline** mengunduh data berdasarkan suatu indeks pada periode waktu tertentu.
* Metode **sys.path.append()** digunakan untuk menambahkan jalur sementara. Dengan demikian, jalur tersebut akan valid untuk sebuah sesi, misalnya.
* Python **itertool** adalah modul yang menyediakan berbagai fungsi yang bekerja pada iterator untuk menghasilkan iterator yang kompleks. Modul ini berfungsi sebagai alat yang cepat dan hemat memori.

In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


<a id='1.4'></a>
## 2.4. Membuat Folder

* **config** berisi pengaturan tanggal periode training dan trading, indikator, hyperparameter setiap model/agen DRL.
* **config_tickers** berisi pengaturan daftar saham yang akan diproses.
* **import os** digunakan untuk meng-import modul, merupakan module pada python agar python itu sendiri berinteraksi langsung terhadap sistem operasi.
* **check_and_make_directories** digunakan untuk mengecek dan membuat folder.


In [4]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

<a id='2'></a>
# Part 3. Mengunduh Data

Yahoo Finance menyediakan data saham, berita keuangan, laporan keuangan, dll, secara gratis.
* FinRL *Library* menggunakan kelas **YahooDownloader** di FinRL-Meta untuk mengambil data melalui Yahoo Finance API
* Batas Pemanggilan: Menggunakan API Publik (tanpa autentikasi), Pengguna dibatasi hingga 2.000 permintaan per jam per IP (atau hingga total 48.000 permintaan per hari).

-----
kelas YahooDownloader:
    Mengambil data saham harian dari
    Yahoo Finance API

    Atribut
    ----------
        start_date : str
            tanggal mulai data
        end_date : str
            tanggal akhir data
        ticker_list : list
            daftar ticker saham (dimodifikasi dari config.py)

    Methods
    -------
    fetch_data()


Penetapan tanggal training dan trading dapat dilakukan dengan mengaturnya pada finrl/config.py atau dengan menetapkannya di sel notebook.

In [5]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2017-12-31'
TRADE_START_DATE = '2018-01-01'
TRADE_END_DATE = '2019-12-20'

In [6]:
#Data diperoleh dari idx.co.id diakses pada tanggal 27/08/2022
#Fraksi 1: <Rp200,00
fraksi1 = ['KIJA.JK', 'LCGP.JK', 'LMPI.JK', 'LPKR.JK']

#Fraksi 2: Rp200,00 - Rp500,00
fraksi2 = ['BMTR.JK','BTON.JK','FORU.JK','GEMA.JK']

#Fraksi 3: Rp500,00 - Rp2.000,00
fraksi3 = ['AKRA.JK','BRPT.JK','KLBF.JK','MEDC.JK']

#Fraksi 4: Rp2000,00 - Rp5000,00
fraksi4 = ['JECC.JK', 'TMAS.JK', 'TPIA.JK', 'UNVR.JK']

#Fraksi 5: >Rp5000,00
fraksi5 =['INCO.JK','INDF.JK','INTP.JK','UNTR.JK']



In [7]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = fraksi3).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (10916, 8)


* **df.shape** digunakan untuk mendapatkan jumlah baris dan kolom

In [8]:
df.shape

(10916, 8)

* **nunique()** mengembalikan jumlah nilai unik untuk setiap kolom. Dengan menentukan sumbu kolom ( axis='columns' ), metode nunique() mencari berdasarkan kolom tersebut dan mengembalikan jumlah nilai unik untuk setiap baris.

In [9]:
df.nunique()

date       2729
open       1657
high       1700
low        1688
close      3586
volume    10043
tic           4
day           5
dtype: int64

* **unique()** digunakan untuk menemukan elemen unik dari array. Mengembalikan elemen unik yang diurutkan dari sebuah array.

In [10]:
print(df['tic'].unique())

['AKRA.JK' 'BRPT.JK' 'KLBF.JK' 'MEDC.JK']


* **describe()** digunakan untuk menampilkan detail statistik dasar seperti persentil, mean, std, dll. Hanya kolom yang bertipe numerik yang akan ditampilkan statistiknya.

In [11]:
df.describe()

,open,high,low,close,volume,day
count,10916.000000,10916.000000,10916.000000,10916.000000,1.091600e+04,10916.000000
mean,676.870151,686.310986,666.324311,586.212896,5.437589e+07,1.988641
std,508.241950,513.534252,501.705070,441.561245,8.763648e+07,1.409951
min,11.800000,12.000000,11.600000,11.511533,0.000000e+00,0.000000
25%,254.000000,258.071457,248.000000,197.278900,1.199262e+07,1.000000
50%,582.857178,593.809570,571.714325,524.874207,3.198650e+07,2.000000
75%,1030.000000,1045.000000,1010.000000,848.782089,6.262312e+07,3.000000
max,1905.000000,1915.000000,1890.000000,1641.142944,1.835330e+09,4.000000


* **sort_values()** berfungsi mengurutkan bingkai data dalam urutan *ascending* atau *descending* dari kolom yang dilewati.
* **ignore_index** berfungsi untuk menentukan untuk mereset indeks mulai dari nol. Secara default disetel *false*.
* **head()** digunakan untuk mendapatkan n baris pertama.

In [12]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2009-01-05,138.000000,144.000000,130.000000,82.180908,2990000,AKRA.JK,0
1,2009-01-05,61.000000,70.000000,60.000000,64.386543,95470000,BRPT.JK,0
2,2009-01-05,81.000000,84.000000,80.000000,63.608833,165165000,KLBF.JK,0
3,2009-01-05,371.428619,382.857178,361.904785,349.914185,20,MEDC.JK,0
4,2009-01-06,126.000000,138.000000,126.000000,80.989883,2915000,AKRA.JK,1


<a id='3'></a>
# Bagian 4: Memproses Data

Periksa data yang hilang dan melakukan *feature engineering* untuk mengubah data menjadi *state*.
* **Menambahkan indikator teknis**

  Dalam praktik jual-beli, berbagai informasi perlu diperhitungkan, seperti harga historis, kepemilikan saham saat ini, indikator teknis, dll. Indikator teknis yang dapat digunakan antara lain, MACD, RSI, CCI, ADX, Bollinger Bands, dll. 
* **Menambahkan indeks turbulensi**

  Risk-aversion mencerminkan seberapa berani investor melakukan jual-beli saham dengan risiko yang tinggi. Hal ini mempengaruhi strategi trading investor ketika menghadapi tingkat volatilitas pasar yang berbeda. Untuk mengendalikan risiko dalam skenario terburuk, seperti krisis keuangan tahun 1998, FinRL menggunakan indeks turbulensi yang mengukur fluktuasi harga aset yang ekstrem.



-----
kelas FeatureEngineer:
    Menyediakan metode untuk preprocessing data harga saham

    Attributes
    ----------
        use_technical_indicator : boolean
            menggunakan indikator teknis atau tidak
        tech_indicator_list : list
            daftar nama indikator teknis (dimodifikasi dari neofinrl_config.py)
        use_vix : boolean
            menggunakan Volatility Index (VIX) atau tidak
        use_turbulence : boolean
            menggunakan indeks turbulensi atau tidak
        user_defined_feature:boolean
            menggunakan fitur yang ditentukan pengguna atau tidak
    Methods
    -------
    preprocess_data()
        metode utama untuk melakukan feature engineering


In [13]:
INDICATORS = [
    "macd",
    "rsi_30",
    "cci_30"
]

Membersihkan data mentah dari nilai-nilai yang hilang. Sehingga saham dengan data yang hilang dihapus dari daftar.

In [14]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (2759, 8)
Successfully added vix
Successfully added turbulence index


In [15]:
processed.describe()

,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
count,10608.000000,10608.000000,10608.000000,10608.000000,1.060800e+04,10608.000000,10608.000000,10608.000000,10608.000000,10608.000000,10608.000000
mean,676.401814,685.871882,665.846480,585.740007,5.464545e+07,2.019231,2.206193,51.516585,6.160626,18.179936,3.947660
std,508.295753,513.616606,501.771123,441.558233,8.806149e+07,1.397923,15.820839,10.124209,117.200928,7.335005,6.891968
min,11.800000,12.000000,11.600000,11.511533,0.000000e+00,0.000000,-68.687701,0.000000,-537.753058,9.140000,0.000000
25%,252.285736,258.000000,247.619080,196.493668,1.219387e+07,1.000000,-4.543439,44.995466,-80.584203,13.287500,0.723494
50%,580.952454,590.476257,571.428650,524.873657,3.219650e+07,2.000000,0.446353,51.189583,4.065761,15.985000,2.058247
75%,1030.000000,1045.000000,1010.000000,848.325500,6.289125e+07,3.000000,8.731651,57.443177,88.948683,20.629999,4.438087
max,1905.000000,1915.000000,1890.000000,1641.142944,1.835330e+09,4.000000,103.154185,100.000000,548.793817,56.650002,149.008488


In [16]:
processed.nunique()

date           2652
open           1645
high           1686
low            1682
close          3548
volume         9784
tic               4
day               5
macd          10602
rsi_30         9094
cci_30        10603
vix            1441
turbulence     2399
dtype: int64

* **tolist()** digunakan untuk mengubah elemen data array menjadi *list*.
* **unique()** digunakan untuk menemukan elemen unik dari array. Mengembalikan elemen unik yang diurutkan dari sebuah array.

In [17]:
list_ticker = processed["tic"].unique().tolist()
list_ticker

['AKRA.JK', 'BRPT.JK', 'KLBF.JK', 'MEDC.JK']

* **date_range()** digunakan untuk mendapatkan frekuensi tetap DatetimeIndex. date_range() digunakan untuk membuat rentang tanggal di pandas.
* **max()** mengembalikan item dengan nilai tertinggi, atau item dengan nilai tertinggi dalam *iterable*. Jika nilainya adalah string, maka dilakukan perbandingan abjad.
* **min()** mengembalikan item dengan nilai terendah, atau item dengan nilai terendah dalam iterable. Jika nilainya adalah string,  maka dilakukan perbandingan abjad.
* **astype()** digunakan untuk mengubah tipe data dari suatu bentuk *series*.

In [18]:
list_date = list(pd.date_range(df['date'].min(),df['date'].max()).astype(str))
list_date

['2009-01-05',
 '2009-01-06',
 '2009-01-07',
 '2009-01-08',
 '2009-01-09',
 '2009-01-10',
 '2009-01-11',
 '2009-01-12',
 '2009-01-13',
 '2009-01-14',
 '2009-01-15',
 '2009-01-16',
 '2009-01-17',
 '2009-01-18',
 '2009-01-19',
 '2009-01-20',
 '2009-01-21',
 '2009-01-22',
 '2009-01-23',
 '2009-01-24',
 '2009-01-25',
 '2009-01-26',
 '2009-01-27',
 '2009-01-28',
 '2009-01-29',
 '2009-01-30',
 '2009-01-31',
 '2009-02-01',
 '2009-02-02',
 '2009-02-03',
 '2009-02-04',
 '2009-02-05',
 '2009-02-06',
 '2009-02-07',
 '2009-02-08',
 '2009-02-09',
 '2009-02-10',
 '2009-02-11',
 '2009-02-12',
 '2009-02-13',
 '2009-02-14',
 '2009-02-15',
 '2009-02-16',
 '2009-02-17',
 '2009-02-18',
 '2009-02-19',
 '2009-02-20',
 '2009-02-21',
 '2009-02-22',
 '2009-02-23',
 '2009-02-24',
 '2009-02-25',
 '2009-02-26',
 '2009-02-27',
 '2009-02-28',
 '2009-03-01',
 '2009-03-02',
 '2009-03-03',
 '2009-03-04',
 '2009-03-05',
 '2009-03-06',
 '2009-03-07',
 '2009-03-08',
 '2009-03-09',
 '2009-03-10',
 '2009-03-11',
 '2009-03-

* **Array** dapat menyimpan elemen hanya dari satu tipe. Sedangkan **list** juga dapat menyimpan elemen dari tipe data yang berbeda.
* **list()** digunakan untuk membuat objek *list*. Objek *list* adalah kumpulan yang berurutan dan dapat diubah.
* **itertools.product()** digunakan untuk mencari produk kartesius dari iterator yang diberikan, outputnya adalah urutan leksikografis.

In [19]:
combination = list(itertools.product(list_date,list_ticker))
combination

[('2009-01-05', 'AKRA.JK'),
 ('2009-01-05', 'BRPT.JK'),
 ('2009-01-05', 'KLBF.JK'),
 ('2009-01-05', 'MEDC.JK'),
 ('2009-01-06', 'AKRA.JK'),
 ('2009-01-06', 'BRPT.JK'),
 ('2009-01-06', 'KLBF.JK'),
 ('2009-01-06', 'MEDC.JK'),
 ('2009-01-07', 'AKRA.JK'),
 ('2009-01-07', 'BRPT.JK'),
 ('2009-01-07', 'KLBF.JK'),
 ('2009-01-07', 'MEDC.JK'),
 ('2009-01-08', 'AKRA.JK'),
 ('2009-01-08', 'BRPT.JK'),
 ('2009-01-08', 'KLBF.JK'),
 ('2009-01-08', 'MEDC.JK'),
 ('2009-01-09', 'AKRA.JK'),
 ('2009-01-09', 'BRPT.JK'),
 ('2009-01-09', 'KLBF.JK'),
 ('2009-01-09', 'MEDC.JK'),
 ('2009-01-10', 'AKRA.JK'),
 ('2009-01-10', 'BRPT.JK'),
 ('2009-01-10', 'KLBF.JK'),
 ('2009-01-10', 'MEDC.JK'),
 ('2009-01-11', 'AKRA.JK'),
 ('2009-01-11', 'BRPT.JK'),
 ('2009-01-11', 'KLBF.JK'),
 ('2009-01-11', 'MEDC.JK'),
 ('2009-01-12', 'AKRA.JK'),
 ('2009-01-12', 'BRPT.JK'),
 ('2009-01-12', 'KLBF.JK'),
 ('2009-01-12', 'MEDC.JK'),
 ('2009-01-13', 'AKRA.JK'),
 ('2009-01-13', 'BRPT.JK'),
 ('2009-01-13', 'KLBF.JK'),
 ('2009-01-13', 'MED

* **Pandas DataFrame** adalah dua dimensi yang dapat berubah ukuran, struktur data tabular yang berpotensi heterogen dengan sumbu berlabel (baris dan kolom).
* **merge()** berfungsi untuk memperbarui konten dua DataFrame dengan menggabungkannya bersama-sama, menggunakan metode yang ditentukan.

In [20]:
processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,AKRA.JK,138.000000,144.000000,130.000000,82.180908,2990000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
1,2009-01-05,BRPT.JK,61.000000,70.000000,60.000000,64.386543,95470000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
2,2009-01-05,KLBF.JK,81.000000,84.000000,80.000000,63.608833,165165000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
3,2009-01-05,MEDC.JK,371.428619,382.857178,361.904785,349.914185,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
4,2009-01-06,AKRA.JK,126.000000,138.000000,126.000000,80.989883,2915000.0,1.0,-0.026722,0.000000,-66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15999,2019-12-18,MEDC.JK,757.671936,783.068787,744.973572,705.821960,138154983.0,2.0,47.405345,65.214848,164.620997,12.580000,1.839519
16000,2019-12-19,AKRA.JK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16001,2019-12-19,BRPT.JK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16002,2019-12-19,KLBF.JK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* **isin()** digunakan untuk memfilter data frames. isin() dapat membantu untuk memilih baris dengan memiliki nilai tertentu (atau beberapa nilai tertentu) dalam kolom tertentu.

In [21]:
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,AKRA.JK,138.000000,144.000000,130.000000,82.180908,2990000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
1,2009-01-05,BRPT.JK,61.000000,70.000000,60.000000,64.386543,95470000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
2,2009-01-05,KLBF.JK,81.000000,84.000000,80.000000,63.608833,165165000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
3,2009-01-05,MEDC.JK,371.428619,382.857178,361.904785,349.914185,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
4,2009-01-06,AKRA.JK,126.000000,138.000000,126.000000,80.989883,2915000.0,1.0,-0.026722,0.000000,-66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,MEDC.JK,800.000000,833.862427,757.671936,721.772766,169035575.0,1.0,46.290195,67.775334,217.871857,12.290000,9.745929
15996,2019-12-18,AKRA.JK,774.000000,778.000000,766.000000,693.404297,66851500.0,2.0,6.737222,52.879110,100.000945,12.580000,1.839519
15997,2019-12-18,BRPT.JK,1445.000000,1480.000000,1435.000000,1469.766479,104822200.0,2.0,87.668064,70.985316,96.525673,12.580000,1.839519
15998,2019-12-18,KLBF.JK,1635.000000,1635.000000,1610.000000,1541.399414,42302800.0,2.0,11.584220,55.676841,166.121505,12.580000,1.839519


In [22]:
processed_full = processed_full.sort_values(['date','tic'])
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,AKRA.JK,138.000000,144.000000,130.000000,82.180908,2990000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
1,2009-01-05,BRPT.JK,61.000000,70.000000,60.000000,64.386543,95470000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
2,2009-01-05,KLBF.JK,81.000000,84.000000,80.000000,63.608833,165165000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
3,2009-01-05,MEDC.JK,371.428619,382.857178,361.904785,349.914185,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
4,2009-01-06,AKRA.JK,126.000000,138.000000,126.000000,80.989883,2915000.0,1.0,-0.026722,0.000000,-66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,MEDC.JK,800.000000,833.862427,757.671936,721.772766,169035575.0,1.0,46.290195,67.775334,217.871857,12.290000,9.745929
15996,2019-12-18,AKRA.JK,774.000000,778.000000,766.000000,693.404297,66851500.0,2.0,6.737222,52.879110,100.000945,12.580000,1.839519
15997,2019-12-18,BRPT.JK,1445.000000,1480.000000,1435.000000,1469.766479,104822200.0,2.0,87.668064,70.985316,96.525673,12.580000,1.839519
15998,2019-12-18,KLBF.JK,1635.000000,1635.000000,1610.000000,1541.399414,42302800.0,2.0,11.584220,55.676841,166.121505,12.580000,1.839519


* **fillna()** digunakan untuk mengisi nilai NA/NaN menggunakan metode yang ditentukan. fillna(0) artinya baris NA/NaN diisi dengan nilai misalnya 0.

In [23]:
processed_full = processed_full.fillna(0)
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,AKRA.JK,138.000000,144.000000,130.000000,82.180908,2990000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
1,2009-01-05,BRPT.JK,61.000000,70.000000,60.000000,64.386543,95470000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
2,2009-01-05,KLBF.JK,81.000000,84.000000,80.000000,63.608833,165165000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
3,2009-01-05,MEDC.JK,371.428619,382.857178,361.904785,349.914185,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
4,2009-01-06,AKRA.JK,126.000000,138.000000,126.000000,80.989883,2915000.0,1.0,-0.026722,0.000000,-66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,MEDC.JK,800.000000,833.862427,757.671936,721.772766,169035575.0,1.0,46.290195,67.775334,217.871857,12.290000,9.745929
15996,2019-12-18,AKRA.JK,774.000000,778.000000,766.000000,693.404297,66851500.0,2.0,6.737222,52.879110,100.000945,12.580000,1.839519
15997,2019-12-18,BRPT.JK,1445.000000,1480.000000,1435.000000,1469.766479,104822200.0,2.0,87.668064,70.985316,96.525673,12.580000,1.839519
15998,2019-12-18,KLBF.JK,1635.000000,1635.000000,1610.000000,1541.399414,42302800.0,2.0,11.584220,55.676841,166.121505,12.580000,1.839519


In [24]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,AKRA.JK,138.000000,144.000000,130.000000,82.180908,2990000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.0
1,2009-01-05,BRPT.JK,61.000000,70.000000,60.000000,64.386543,95470000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.0
2,2009-01-05,KLBF.JK,81.000000,84.000000,80.000000,63.608833,165165000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.0
3,2009-01-05,MEDC.JK,371.428619,382.857178,361.904785,349.914185,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.0
4,2009-01-06,AKRA.JK,126.000000,138.000000,126.000000,80.989883,2915000.0,1.0,-0.026722,0.000000,-66.666667,38.560001,0.0
5,2009-01-06,BRPT.JK,66.000000,71.000000,62.000000,60.484322,72175000.0,1.0,-0.087550,0.000000,-66.666667,38.560001,0.0
6,2009-01-06,KLBF.JK,81.000000,82.000000,80.000000,63.608833,44137500.0,1.0,0.000000,0.000000,-66.666667,38.560001,0.0
7,2009-01-06,MEDC.JK,373.333374,380.952423,361.904785,342.736542,24719622.0,1.0,-0.161037,0.000000,-66.666667,38.560001,0.0
8,2009-01-07,AKRA.JK,136.000000,142.000000,130.000000,80.989883,12010000.0,2.0,-0.034049,0.000000,25.058460,43.389999,0.0
9,2009-01-07,BRPT.JK,63.000000,70.000000,61.000000,61.459877,80160000.0,2.0,-0.081304,20.547945,-100.000000,43.389999,0.0


In [25]:
processed_full.nunique()

date           2652
tic               4
open           1645
high           1686
low            1682
close          3548
volume         9784
day               5
macd          10602
rsi_30         9094
cci_30        10603
vix            1441
turbulence     2399
dtype: int64

* **info()** berfungsi untuk mencetak informasi tentang DataFrame. Informasi tersebut berisi jumlah kolom, label kolom, tipe data kolom, penggunaan memori, indeks rentang, dan jumlah sel di setiap kolom (nilai bukan nol).

In [26]:
processed_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10608 entries, 0 to 15999
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        10608 non-null  object 
 1   tic         10608 non-null  object 
 2   open        10608 non-null  float64
 3   high        10608 non-null  float64
 4   low         10608 non-null  float64
 5   close       10608 non-null  float64
 6   volume      10608 non-null  float64
 7   day         10608 non-null  float64
 8   macd        10608 non-null  float64
 9   rsi_30      10608 non-null  float64
 10  cci_30      10608 non-null  float64
 11  vix         10608 non-null  float64
 12  turbulence  10608 non-null  float64
dtypes: float64(11), object(2)
memory usage: 1.1+ MB


* **to_csv()** mengonversi DataFrame menjadi data CSV/comma separated value (nilai yang dipisahkan koma). Kita dapat melewatkan objek file untuk menulis data CSV ke dalam file. Jika tidak, data CSV dikembalikan dalam format string.
* **loc** berbasis label, yang berarti baris dan kolom diambil berdasarkan label baris dan kolomnya. **iloc** berbasis posisi integer, maka baris dan kolom dapat diambil dengan menggunakan nilai posisi integernya (posisi integer berbasis 0).

In [27]:
#Save the dataset by the tic filter
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  df_temp.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/DatasetsPerTic/Fraksi3/'+ stock +'.csv',index=False)
  display(df_temp)

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,AKRA.JK,138.0,144.0,130.0,82.180908,2990000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
4,2009-01-06,AKRA.JK,126.0,138.0,126.0,80.989883,2915000.0,1.0,-0.026722,0.000000,-66.666667,38.560001,0.000000
8,2009-01-07,AKRA.JK,136.0,142.0,130.0,80.989883,12010000.0,2.0,-0.034049,0.000000,25.058460,43.389999,0.000000
12,2009-01-08,AKRA.JK,136.0,136.0,132.0,80.989883,237500.0,3.0,-0.036547,0.000000,-31.558217,42.560001,0.000000
16,2009-01-09,AKRA.JK,136.0,136.0,132.0,80.989883,62500.0,4.0,-0.037135,0.000000,-28.828855,42.820000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,AKRA.JK,754.0,756.0,740.0,672.013977,36956500.0,3.0,-1.634581,49.494913,40.825594,13.940000,0.220926
15976,2019-12-13,AKRA.JK,758.0,770.0,754.0,684.491699,58397500.0,4.0,0.871302,51.475430,75.206993,12.630000,4.513726
15988,2019-12-16,AKRA.JK,768.0,778.0,756.0,689.839233,62960000.0,0.0,3.251256,52.304633,89.005454,12.140000,14.500617
15992,2019-12-17,AKRA.JK,774.0,778.0,766.0,689.839233,53565500.0,1.0,5.078840,52.304633,97.518591,12.290000,9.745929


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
1,2009-01-05,BRPT.JK,61.0,70.0,60.0,64.386543,95470000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
5,2009-01-06,BRPT.JK,66.0,71.0,62.0,60.484322,72175000.0,1.0,-0.087550,0.000000,-66.666667,38.560001,0.000000
9,2009-01-07,BRPT.JK,63.0,70.0,61.0,61.459877,80160000.0,2.0,-0.081304,20.547945,-100.000000,43.389999,0.000000
13,2009-01-08,BRPT.JK,64.0,65.0,61.0,62.435429,29180000.0,3.0,-0.041249,34.475997,-133.333333,42.560001,0.000000
17,2009-01-09,BRPT.JK,62.0,66.0,62.0,60.484322,42590000.0,4.0,-0.096254,25.299898,-82.652044,42.820000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,BRPT.JK,1430.0,1435.0,1405.0,1400.250610,58912700.0,3.0,95.708893,68.859335,92.321350,13.940000,0.220926
15977,2019-12-13,BRPT.JK,1415.0,1420.0,1380.0,1410.181396,75925900.0,4.0,93.046575,69.276538,83.889991,12.630000,4.513726
15989,2019-12-16,BRPT.JK,1410.0,1430.0,1405.0,1395.285156,74485300.0,0.0,88.712049,67.865673,81.687791,12.140000,14.500617
15993,2019-12-17,BRPT.JK,1405.0,1435.0,1380.0,1425.077637,72110900.0,1.0,86.681698,69.164937,78.838577,12.290000,9.745929


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
2,2009-01-05,KLBF.JK,81.0,84.0,80.0,63.608833,165165000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
6,2009-01-06,KLBF.JK,81.0,82.0,80.0,63.608833,44137500.0,1.0,0.000000,0.000000,-66.666667,38.560001,0.000000
10,2009-01-07,KLBF.JK,81.0,83.0,80.0,63.608833,166020000.0,2.0,0.000000,20.547945,0.000000,43.389999,0.000000
14,2009-01-08,KLBF.JK,81.0,81.0,80.0,63.608833,59210000.0,3.0,0.000000,34.475997,-100.000000,42.560001,0.000000
18,2009-01-09,KLBF.JK,81.0,96.0,81.0,74.602966,397677500.0,4.0,0.437482,100.000000,166.666667,42.820000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,KLBF.JK,1550.0,1590.0,1545.0,1480.120605,26446200.0,3.0,-0.613375,49.672440,42.118931,13.940000,0.220926
15978,2019-12-13,KLBF.JK,1600.0,1600.0,1575.0,1508.403076,18588700.0,4.0,2.470021,52.781494,100.013048,12.630000,4.513726
15990,2019-12-16,KLBF.JK,1605.0,1610.0,1565.0,1484.834351,16279800.0,0.0,2.977512,50.112717,80.762455,12.140000,14.500617
15994,2019-12-17,KLBF.JK,1585.0,1635.0,1555.0,1541.399414,46622200.0,1.0,7.853500,55.676841,134.244489,12.290000,9.745929


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
3,2009-01-05,MEDC.JK,371.428619,382.857178,361.904785,349.914185,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
7,2009-01-06,MEDC.JK,373.333374,380.952423,361.904785,342.736542,24719622.0,1.0,-0.161037,0.000000,-66.666667,38.560001,0.000000
11,2009-01-07,MEDC.JK,367.619080,386.666718,365.714325,346.325378,20.0,2.0,-0.093903,34.091005,77.237628,43.389999,0.000000
15,2009-01-08,MEDC.JK,360.000031,361.904785,354.285736,339.147644,20.0,3.0,-0.320218,19.990771,-133.333333,42.560001,0.000000
19,2009-01-09,MEDC.JK,365.714325,369.523834,358.095276,344.530945,20.0,4.0,-0.222238,39.428291,-43.416331,42.820000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,MEDC.JK,698.412720,715.343933,681.481506,661.957336,87890315.0,3.0,28.039621,66.014572,205.825025,13.940000,0.220926
15979,2019-12-13,MEDC.JK,715.343933,744.973572,711.111084,693.858887,113998066.0,4.0,33.547092,69.154002,219.392372,12.630000,4.513726
15991,2019-12-16,MEDC.JK,740.740723,804.232788,736.507935,757.661987,225414866.0,0.0,42.569465,74.103417,248.805834,12.140000,14.500617
15995,2019-12-17,MEDC.JK,800.000000,833.862427,757.671936,721.772766,169035575.0,1.0,46.290195,67.775334,217.871857,12.290000,9.745929


In [28]:
#memperbarui dataframe dengan bentuk harga per lot

processed_full_lot = processed_full[['open','high','low','close']].mul(100)

#drop some columns
processed_full_noOHLC = processed_full.drop(['open','high','low','close'], axis=1)
  
processed_full_lot = pd.concat([processed_full_noOHLC, processed_full_lot], axis=1)

processed_full_lot = processed_full_lot[['date', 'tic', 'open', 'high', 'low', 'close', 'volume', 'day', 'macd', 'rsi_30', 'cci_30', 'vix', 'turbulence']]

processed_full_lot

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,AKRA.JK,13800.000000,14400.000000,13000.000000,8218.090820,2990000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
1,2009-01-05,BRPT.JK,6100.000000,7000.000000,6000.000000,6438.654327,95470000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
2,2009-01-05,KLBF.JK,8100.000000,8400.000000,8000.000000,6360.883331,165165000.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
3,2009-01-05,MEDC.JK,37142.861938,38285.717773,36190.478516,34991.418457,20.0,0.0,0.000000,0.000000,-66.666667,39.080002,0.000000
4,2009-01-06,AKRA.JK,12600.000000,13800.000000,12600.000000,8098.988342,2915000.0,1.0,-0.026722,0.000000,-66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,MEDC.JK,80000.000000,83386.242676,75767.193604,72177.276611,169035575.0,1.0,46.290195,67.775334,217.871857,12.290000,9.745929
15996,2019-12-18,AKRA.JK,77400.000000,77800.000000,76600.000000,69340.429688,66851500.0,2.0,6.737222,52.879110,100.000945,12.580000,1.839519
15997,2019-12-18,BRPT.JK,144500.000000,148000.000000,143500.000000,146976.647949,104822200.0,2.0,87.668064,70.985316,96.525673,12.580000,1.839519
15998,2019-12-18,KLBF.JK,163500.000000,163500.000000,161000.000000,154139.941406,42302800.0,2.0,11.584220,55.676841,166.121505,12.580000,1.839519


<a id='4'></a>
# Bagian 5. Membangun Lingkungan Jual-Beli Saham dengan OpenAI Gym-style
Proses *training* meliputi mengamati perubahan harga saham, mengambil tindakan dan perhitungan *reward*. Dengan berinteraksi dengan lingkungan pasar, agen pada akhirnya akan memperoleh strategi perdagangan yang dapat memaksimalkan imbalan.

*Environment* pasar dibangun  menggunakan OpenAI Gym yang mensimulasikan pasar saham dengan data historis pasar.

<a id='4.1'></a>
## 5.1. Memisahkan Data
Data dibagi menjadi dua bagian, yaitu bagian pelatihan dan bagian pengujian dengan keterangan sebagai berikut:

Periode data pelatihan: 2009-01-01 hingga 2017-12-31

Periode data perdagangan: 2018-01-01 hingga 2022-08-15

* **data_split** membagi data menjadi himpunan *train, test,* dan/atau *validation.*
* **len()** mengembalikan jumlah item dalam suatu objek. Namun, jika objek adalah string, maka fungsi len() akan mengembalikan jumlah karakter dalam string.
* **tail()** digunakan untuk mendapatkan n baris terakhir.

In [29]:
train = data_split(processed_full_lot, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full_lot, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

8628
1980


In [30]:
train.tail()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
2155,2017-12-28,MEDC.JK,76190.478516,77037.036133,74074.072266,70980.963135,57711622.0,3.0,17.428146,59.931074,74.238054,10.18,1.060699
2156,2017-12-29,AKRA.JK,129000.000000,129000.000000,127000.000000,103281.579590,17576000.0,4.0,-13.380607,45.158695,10.627991,11.04,0.497414
2156,2017-12-29,BRPT.JK,45200.000000,45800.000000,44200.000000,44095.022583,141907000.0,4.0,2.984996,53.936636,-35.182449,11.04,0.497414
2156,2017-12-29,KLBF.JK,167000.000000,170000.000000,167000.000000,153249.194336,40084400.0,4.0,9.240524,54.156941,136.739070,11.04,0.497414
2156,2017-12-29,MEDC.JK,75343.914795,76190.478516,73650.793457,70980.963135,38584586.0,4.0,16.804936,59.931074,56.067394,11.04,0.497414


In [31]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2018-01-02,AKRA.JK,127500.000000,128500.000000,123500.000000,100841.870117,25448000.0,1.0,-12.741437,42.797321,-27.402704,9.77,4.977856
0,2018-01-02,BRPT.JK,45400.000000,45400.000000,44000.000000,43899.908447,110809000.0,1.0,1.533834,53.320545,-60.693822,9.77,4.977856
0,2018-01-02,KLBF.JK,169000.000000,173000.000000,167500.000000,156876.391602,17137900.0,1.0,13.548843,58.402810,189.233758,9.77,4.977856
0,2018-01-02,MEDC.JK,75343.914795,77037.036133,75343.914795,71379.736328,45424968.0,1.0,15.726914,60.442577,79.238661,9.77,4.977856
1,2018-01-03,AKRA.JK,126000.000000,126500.000000,123000.000000,100435.260010,34013000.0,2.0,-13.495459,42.414932,-48.046638,9.15,6.195140


In [32]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 4, State Space: 21


Transaction Fee
    # BUY Fee => 0.23% (Broker Fee(0.19%) + Levy(0.04%))
    # SELL Fee => 0.54% (Broker Fee(0.29%) + Levy(0.04%) + PPN(0.11%) + PPh(0.1%))

In [33]:
buy_cost_list = [0.0023] * stock_dimension
sell_cost_list = [0.0054] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 100000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

<a id='4.2'></a>
## 5.2. Lingkungan untuk Pelatihan *(Training)*



In [34]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Bagian 6: Melatih Agen DRL
* Algoritma DRL dibangun dengan menggunakan **Stable Baselines 3**. 

* FinRL mencakup algoritma DRL standar yang disempurnakan, seperti DQN, DDPG, DDPG Multi-Agen, PPO, SAC, A2C, dan TD3. FinRL juga mengizinkan pengguna untuk merancang algoritma DRL sendiri dengan mengadaptasi algoritma DRL yang telah disebutkan.



-----
kelas DRLAgent:
    Menyediakan implementasi untuk algoritma DRL

    Atribut
    ----------
        env: gym environment class
            kelas yang ditentukan pengguna

    Methods
    -------
        get_model()
            mengatur algoritma DRL
        train_model()
            melatih algoritma DRL dalam set data latih dan mengeluarkan model yang dilatih
        DRL_prediction()
            membuat prediksi dalam kumpulan data pengujian dan dapatkan hasil


In [35]:
agent = DRLAgent(env = env_train)

**Agent Training: 5 algorithms (A2C, PPO, TD3, SAC, DDPG)**

<a id='5.1'></a>
##6.1. Agen 1: A2C


In [36]:
A2C_PARAMS = {
    "n_steps": 150, 
    "ent_coef": 0.0001, 
    "learning_rate": 0.0014,
}
model_a2c = agent.get_model("a2c",model_kwargs = A2C_PARAMS)

{'n_steps': 150, 'ent_coef': 0.0001, 'learning_rate': 0.0014}
Using cuda device


In [37]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000)

-------------------------------------
| time/                 |           |
|    fps                | 348       |
|    iterations         | 100       |
|    time_elapsed       | 43        |
|    total_timesteps    | 15000     |
| train/                |           |
|    entropy_loss       | -5.58     |
|    explained_variance | 4.47e-06  |
|    learning_rate      | 0.0014    |
|    n_updates          | 99        |
|    policy_loss        | -7.45e+03 |
|    reward             | 679.4756  |
|    std                | 0.978     |
|    value_loss         | 1.06e+08  |
-------------------------------------
day: 2156, episode: 10
begin_total_asset: 100000000.00
end_total_asset: 1264920840.54
total_reward: 1164920840.54
total_cost: 39056259.04
total_trades: 6541
Sharpe: 1.114
-------------------------------------
| time/                 |           |
|    fps                | 361       |
|    iterations         | 200       |
|    time_elapsed       | 82        |
|    total_timesteps    | 30000

<a id='5.2'></a>
##6.2. Agen 2: PPO

In [38]:
PPO_PARAMS = {
    "n_steps": 50000,
    "ent_coef": 0.7,
    "learning_rate": 0.0016,
    "batch_size": 1750,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

{'n_steps': 50000, 'ent_coef': 0.7, 'learning_rate': 0.0016, 'batch_size': 1750}
Using cuda device


/usr/local/lib/python3.7/dist-packages/stable_baselines3/ppo/ppo.py:147: UserWarning: You have specified a mini-batch size of 1750, but because the `RolloutBuffer` is of size `n_steps * n_envs = 50000`, after every 28 untruncated mini-batches, there will be a truncated mini-batch of size 1000
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=50000 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"


In [39]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000)

day: 2156, episode: 30
begin_total_asset: 100000000.00
end_total_asset: 335699370.13
total_reward: 235699370.13
total_cost: 100446431.82
total_trades: 8420
Sharpe: 0.731
day: 2156, episode: 40
begin_total_asset: 100000000.00
end_total_asset: 84200471.25
total_reward: -15799528.75
total_cost: 88388666.11
total_trades: 8251
Sharpe: 0.047
-----------------------------------
| time/              |            |
|    fps             | 376        |
|    iterations      | 1          |
|    time_elapsed    | 132        |
|    total_timesteps | 50000      |
| train/             |            |
|    reward          | -20.972778 |
-----------------------------------


<a id='5.3'></a>
##6.3. Agen 3: TD3

In [40]:
TD3_PARAMS = {
    "batch_size": 7000, 
    "buffer_size": 10000, 
    "learning_rate": 0.9  
}
model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

{'batch_size': 7000, 'buffer_size': 10000, 'learning_rate': 0.9}
Using cuda device


In [41]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=50000)

day: 2156, episode: 50
begin_total_asset: 100000000.00
end_total_asset: 108380515.92
total_reward: 8380515.92
total_cost: 16553717.47
total_trades: 2815
Sharpe: 0.207
---------------------------------
| time/              |          |
|    episodes        | 4        |
|    fps             | 98       |
|    time_elapsed    | 87       |
|    total_timesteps | 8628     |
| train/             |          |
|    actor_loss      | 8.68e+06 |
|    critic_loss     | 9.62e+13 |
|    learning_rate   | 0.9      |
|    n_updates       | 6471     |
|    reward          | 0.0      |
---------------------------------
---------------------------------
| time/              |          |
|    episodes        | 8        |
|    fps             | 86       |
|    time_elapsed    | 199      |
|    total_timesteps | 17256    |
| train/             |          |
|    actor_loss      | 5.91e+06 |
|    critic_loss     | 3.54e+13 |
|    learning_rate   | 0.9      |
|    n_updates       | 15099    |
|    reward      

<a id='5.4'></a>
##6.4. Agen 4: SAC

In [42]:
SAC_PARAMS = {
    "ent_coef": 1.25,
    "learning_rate": 0.0001,
    "batch_size": 4000,
    "buffer_size": 5000,
    "learning_starts": 14,
    
}
model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

{'ent_coef': 1.25, 'learning_rate': 0.0001, 'batch_size': 4000, 'buffer_size': 5000, 'learning_starts': 14}
Using cuda device


In [43]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=30000)

-----------------------------------
| time/              |            |
|    episodes        | 4          |
|    fps             | 69         |
|    time_elapsed    | 124        |
|    total_timesteps | 8628       |
| train/             |            |
|    actor_loss      | 8.79e+03   |
|    critic_loss     | 6.72e+05   |
|    ent_coef        | 1.25       |
|    learning_rate   | 0.0001     |
|    n_updates       | 8613       |
|    reward          | -255.97157 |
-----------------------------------
day: 2156, episode: 80
begin_total_asset: 100000000.00
end_total_asset: 1018247112.66
total_reward: 918247112.66
total_cost: 229472.19
total_trades: 6468
Sharpe: 1.064
-----------------------------------
| time/              |            |
|    episodes        | 8          |
|    fps             | 69         |
|    time_elapsed    | 249        |
|    total_timesteps | 17256      |
| train/             |            |
|    actor_loss      | 8.43e+03   |
|    critic_loss     | 3.53e+05   |
|   

<a id='5.5'></a>
##6.5. Agen 5: DDPG

In [44]:
DDPG_PARAMS = {
    "batch_size": 1250, 
    "buffer_size": 50000, 
    "learning_rate": 0.0003
}
model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

{'batch_size': 1250, 'buffer_size': 50000, 'learning_rate': 0.0003}
Using cuda device


In [45]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=60000)

day: 2156, episode: 90
begin_total_asset: 100000000.00
end_total_asset: 207051803.94
total_reward: 107051803.94
total_cost: 329036.21
total_trades: 2207
Sharpe: 0.412
---------------------------------
| time/              |          |
|    episodes        | 4        |
|    fps             | 130      |
|    time_elapsed    | 65       |
|    total_timesteps | 8628     |
| train/             |          |
|    actor_loss      | -1.6e+03 |
|    critic_loss     | 4.15e+07 |
|    learning_rate   | 0.0003   |
|    n_updates       | 6471     |
|    reward          | 0.0      |
---------------------------------
---------------------------------
| time/              |          |
|    episodes        | 8        |
|    fps             | 115      |
|    time_elapsed    | 148      |
|    total_timesteps | 17256    |
| train/             |          |
|    actor_loss      | 1.43e+03 |
|    critic_loss     | 2.64e+06 |
|    learning_rate   | 0.0003   |
|    n_updates       | 15099    |
|    reward      

<a id='6'></a>
# Bagian 7: Melakukan Jual-Beli

<a id='6.1'></a>
## 7.1. Performa di Dalam Sampel

Asumsikan modal awal adalah Rp10.000.000.

Pengaturan ambang turbulensi *(turbulence threshold)*

Atur ambang turbulensi menjadi lebih besar dari maksimum dalam data turbulensi sampel. Jika indeks turbulensi saat ini lebih besar dari ambang batas, maka dapat diasumsikan bahwa pasar pada periode tersebut sedang bergejolak

* **drop_duplicates()** menghapus baris duplikat.
* **quantile()** digunakan untuk menghitung kuantil nilai dalam sumbu tertentu. Sumbu default adalah baris.

In [46]:
data_risk_indicator = processed_full[(processed_full.date<TRAIN_END_DATE) & (processed_full.date>=TRAIN_START_DATE)]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [47]:
insample_risk_indicator.vix.describe()

count    2157.000000
mean       18.666583
std         7.813415
min         9.140000
25%        13.370000
50%        16.299999
75%        21.540001
max        56.650002
Name: vix, dtype: float64

In [48]:
insample_risk_indicator.vix.quantile(0.996)

48.84047967529307

In [49]:
insample_risk_indicator.turbulence.describe()

count    2157.000000
mean        3.808147
std         7.015112
min         0.000000
25%         0.656752
50%         1.918114
75%         4.171936
max       149.008488
Name: turbulence, dtype: float64

In [50]:
insample_risk_indicator.turbulence.quantile(0.996)

36.23674085832022

<a id='6.2'></a>
##7.2. Performa di Luar Sampel

FinRL dapat diatur dengan cara melakukan pelatihan ulang secara berkala, misalnya, pelatihan ulang setiap tiga bulan, bulanan, atau mingguan. 

Pada notebook ini, *hyperparameter* hanya diatur satu kali dengan menggunakan data sampel selama periode yang telah ditetapkan. Sehingga, terjadi beberapa peluruhan alfa seiring dengan perpanjangan tanggal perdagangan.

Banyak *hyperparameter* – mis. learning rate, mempengaruhi proses pembelajaran dan biasanya ditentukan dengan menguji beberapa variasi.

In [51]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70, risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [52]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2018-01-02,AKRA.JK,127500.000000,128500.000000,123500.000000,100841.870117,25448000.0,1.0,-12.741437,42.797321,-27.402704,9.77,4.977856
0,2018-01-02,BRPT.JK,45400.000000,45400.000000,44000.000000,43899.908447,110809000.0,1.0,1.533834,53.320545,-60.693822,9.77,4.977856
0,2018-01-02,KLBF.JK,169000.000000,173000.000000,167500.000000,156876.391602,17137900.0,1.0,13.548843,58.402810,189.233758,9.77,4.977856
0,2018-01-02,MEDC.JK,75343.914795,77037.036133,75343.914795,71379.736328,45424968.0,1.0,15.726914,60.442577,79.238661,9.77,4.977856
1,2018-01-03,AKRA.JK,126000.000000,126500.000000,123000.000000,100435.260010,34013000.0,2.0,-13.495459,42.414932,-48.046638,9.15,6.195140


<a id='6.3'></a>
##7.3. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi

<a id='6.3.1'></a>
###7.3.1. Agen 1: A2C

In [53]:
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_a2c, 
    environment = e_trade_gym)

hit end!


In [54]:
df_actions_and_account_value_a2c = pd.merge(df_actions_a2c, df_account_value_a2c, on='date', how='outer')

#adding new column
df_actions_and_account_value_a2c['total_shares'] = ""

#count the number of column of tics
numberOfColumns = len(df_actions_and_account_value_a2c.columns)
numberOfTic = numberOfColumns-2

#selecting column of actions in every tics
action_columns_a2c = df_actions_and_account_value_a2c.iloc[:,1:numberOfTic]

# sum each row
df_actions_and_account_value_a2c['total_shares'] = action_columns_a2c.sum(axis = 1)

#cumulative sum of total_share
df_actions_and_account_value_a2c['total_shares'] = df_actions_and_account_value_a2c['total_shares'].cumsum(skipna=False)

In [55]:
#define function to swap columns
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

#swap points and rebounds columns
df_actions_and_account_value_a2c = swap_columns(df_actions_and_account_value_a2c, 'total_shares','account_value')

#Replace NaN value to 0
df_actions_and_account_value_a2c=df_actions_and_account_value_a2c.fillna(0)

#view updated DataFrame
display(df_actions_and_account_value_a2c)

,date,AKRA.JK,BRPT.JK,KLBF.JK,MEDC.JK,total_shares,account_value
0,2018-01-02,0.0,0.0,46.0,23.0,69.0,1.000000e+08
1,2018-01-03,0.0,0.0,46.0,23.0,138.0,9.982446e+07
2,2018-01-04,0.0,0.0,46.0,23.0,207.0,1.002399e+08
3,2018-01-05,0.0,0.0,46.0,23.0,276.0,1.007098e+08
4,2018-01-08,0.0,0.0,46.0,23.0,345.0,1.012161e+08
...,...,...,...,...,...,...,...
490,2019-12-12,0.0,0.0,0.0,0.0,1004.0,6.722608e+07
491,2019-12-13,0.0,0.0,0.0,0.0,1004.0,6.895085e+07
492,2019-12-16,0.0,0.0,0.0,0.0,1004.0,7.090821e+07
493,2019-12-17,0.0,0.0,0.0,0.0,1004.0,7.003610e+07


In [56]:
df_actions_and_account_value_a2c.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModel/Fraksi3/df_actions_and_account_value_a2c(4).csv',index=False)

<a id='6.3.2'></a>
###7.3.2. Agen 2: PPO

In [57]:
df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
    model=trained_ppo, 
    environment = e_trade_gym)

hit end!


In [58]:
df_actions_and_account_value_ppo = pd.merge(df_actions_ppo, df_account_value_ppo, on='date', how='outer')

#adding new column
df_actions_and_account_value_ppo['total_shares'] = ""

#selecting column of actions in every tics
action_columns_ppo = df_actions_and_account_value_ppo.iloc[:,1:numberOfTic]

# sum each row
df_actions_and_account_value_ppo['total_shares'] = action_columns_ppo.sum(axis = 1)

#cumulative sum of total_share
df_actions_and_account_value_ppo['total_shares'] = df_actions_and_account_value_ppo['total_shares'].cumsum(skipna=False)

In [59]:
#swap points and rebounds columns
df_actions_and_account_value_ppo = swap_columns(df_actions_and_account_value_ppo, 'account_value', 'total_shares')

#Replace NaN value to 0
df_actions_and_account_value_ppo=df_actions_and_account_value_ppo.fillna(0)

#view updated DataFrame
display(df_actions_and_account_value_ppo)

,date,AKRA.JK,BRPT.JK,KLBF.JK,MEDC.JK,total_shares,account_value
0,2018-01-02,0.0,1.0,19.0,0.0,20.0,1.000000e+08
1,2018-01-03,0.0,1.0,19.0,0.0,40.0,9.993060e+07
2,2018-01-04,0.0,1.0,19.0,0.0,60.0,1.000926e+08
3,2018-01-05,0.0,1.0,19.0,0.0,80.0,1.001660e+08
4,2018-01-08,0.0,1.0,19.0,0.0,100.0,1.003305e+08
...,...,...,...,...,...,...,...
490,2019-12-12,0.0,0.0,0.0,0.0,674.0,9.950485e+07
491,2019-12-13,0.0,0.0,0.0,0.0,674.0,1.013487e+08
492,2019-12-16,0.0,0.0,0.0,0.0,674.0,9.978965e+07
493,2019-12-17,0.0,0.0,0.0,0.0,674.0,1.035111e+08


In [60]:
df_actions_and_account_value_ppo.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModel/Fraksi3/df_actions_and_account_value_ppo(4).csv',index=False)

<a id='6.3.3'></a>
###7.3.3. Agen 3: TD3

In [61]:
df_account_value_td3, df_actions_td3 = DRLAgent.DRL_prediction(
    model=trained_td3, 
    environment = e_trade_gym)

hit end!


In [62]:
df_actions_and_account_value_td3 = pd.merge(df_actions_td3, df_account_value_td3, on='date', how='outer')

#adding new column
df_actions_and_account_value_td3['total_shares'] = ""

#selecting column of actions in every tics
action_columns_td3 = df_actions_and_account_value_td3.iloc[:,1:numberOfTic]

# sum each row
df_actions_and_account_value_td3['total_shares'] = action_columns_td3.sum(axis = 1)

#cumulative sum of total_share
df_actions_and_account_value_td3['total_shares'] = df_actions_and_account_value_td3['total_shares'].cumsum(skipna=False)

In [63]:
#swap points and rebounds columns
df_actions_and_account_value_td3 = swap_columns(df_actions_and_account_value_td3, 'account_value', 'total_shares')

#Replace NaN value to 0
df_actions_and_account_value_td3=df_actions_and_account_value_td3.fillna(0)

#view updated DataFrame
display(df_actions_and_account_value_td3)

,date,AKRA.JK,BRPT.JK,KLBF.JK,MEDC.JK,total_shares,account_value
0,2018-01-02,0.0,100.0,0.0,100.0,200.0,1.000000e+08
1,2018-01-03,0.0,100.0,0.0,100.0,400.0,9.971899e+07
2,2018-01-04,0.0,100.0,0.0,100.0,600.0,9.942161e+07
3,2018-01-05,0.0,100.0,0.0,100.0,800.0,1.010046e+08
4,2018-01-08,0.0,100.0,0.0,100.0,1000.0,1.013782e+08
...,...,...,...,...,...,...,...
490,2019-12-12,0.0,0.0,0.0,0.0,1656.0,1.687853e+08
491,2019-12-13,0.0,0.0,0.0,0.0,1656.0,1.723084e+08
492,2019-12-16,0.0,0.0,0.0,0.0,1656.0,1.765704e+08
493,2019-12-17,0.0,0.0,0.0,0.0,1656.0,1.758882e+08


In [64]:
df_actions_and_account_value_td3.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModel/Fraksi3/df_actions_and_account_value_td3(4).csv',index=False)

<a id='6.3.4'></a>
###7.3.4. Agen 4: SAC

In [65]:
df_account_value_sac, df_actions_sac = DRLAgent.DRL_prediction(
    model=trained_sac, 
    environment = e_trade_gym)

hit end!


In [66]:
df_actions_and_account_value_sac = pd.merge(df_actions_sac, df_account_value_sac, on='date', how='outer')

#adding new column
df_actions_and_account_value_sac['total_shares'] = ""

#selecting column of actions in every tics
action_columns_sac = df_actions_and_account_value_sac.iloc[:,1:numberOfTic]

# sum each row
df_actions_and_account_value_sac['total_shares'] = action_columns_sac.sum(axis = 1)

#cumulative sum of total_share
df_actions_and_account_value_sac['total_shares'] = df_actions_and_account_value_sac['total_shares'].cumsum(skipna=False)

In [67]:
#swap points and rebounds columns
df_actions_and_account_value_sac = swap_columns(df_actions_and_account_value_sac, 'account_value', 'total_shares')

#Replace NaN value to 0
df_actions_and_account_value_sac=df_actions_and_account_value_sac.fillna(0)

#view updated DataFrame
display(df_actions_and_account_value_sac)

,date,AKRA.JK,BRPT.JK,KLBF.JK,MEDC.JK,total_shares,account_value
0,2018-01-02,100.0,100.0,100.0,0.0,300.0,1.000000e+08
1,2018-01-03,100.0,100.0,100.0,0.0,600.0,9.935796e+07
2,2018-01-04,100.0,100.0,100.0,0.0,900.0,9.992683e+07
3,2018-01-05,0.0,1.0,63.0,0.0,964.0,1.005588e+08
4,2018-01-08,0.0,0.0,0.0,0.0,964.0,1.025196e+08
...,...,...,...,...,...,...,...
490,2019-12-12,0.0,0.0,0.0,0.0,964.0,1.160527e+08
491,2019-12-13,0.0,0.0,0.0,0.0,964.0,1.177527e+08
492,2019-12-16,0.0,0.0,0.0,0.0,964.0,1.166092e+08
493,2019-12-17,0.0,0.0,0.0,0.0,964.0,1.195592e+08


In [68]:
df_actions_and_account_value_sac.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModel/Fraksi3/df_actions_and_account_value_sac(3).csv',index=False)

<a id='6.3.5'></a>
###7.3.5. Agen 5: DDPG

In [69]:
df_account_value_ddpg, df_actions_ddpg = DRLAgent.DRL_prediction(
    model=trained_ddpg, 
    environment = e_trade_gym)

hit end!


In [70]:
df_actions_and_account_value_ddpg = pd.merge(df_actions_ddpg, df_account_value_ddpg, on='date', how='outer')

#adding new column
df_actions_and_account_value_ddpg['total_shares'] = ""

#selecting column of actions in every tics
action_columns_ddpg = df_actions_and_account_value_ddpg.iloc[:,1:numberOfTic]

# sum each row
df_actions_and_account_value_ddpg['total_shares'] = action_columns_ddpg.sum(axis = 1)

#cumulative sum of total_share
df_actions_and_account_value_ddpg['total_shares'] = df_actions_and_account_value_ddpg['total_shares'].cumsum(skipna=False)

In [71]:
#swap points and rebounds columns
df_actions_and_account_value_ddpg = swap_columns(df_actions_and_account_value_ddpg, 'account_value', 'total_shares')

#Replace NaN value to 0
df_actions_and_account_value_ddpg=df_actions_and_account_value_ddpg.fillna(0)

#view updated DataFrame
display(df_actions_and_account_value_ddpg)

,date,AKRA.JK,BRPT.JK,KLBF.JK,MEDC.JK,total_shares,account_value
0,2018-01-02,0.0,100.0,0.0,100.0,200.0,1.000000e+08
1,2018-01-03,0.0,100.0,0.0,100.0,400.0,9.971899e+07
2,2018-01-04,0.0,100.0,0.0,100.0,600.0,9.942161e+07
3,2018-01-05,0.0,100.0,0.0,100.0,800.0,1.010046e+08
4,2018-01-08,0.0,100.0,0.0,100.0,1000.0,1.013782e+08
...,...,...,...,...,...,...,...
490,2019-12-12,0.0,0.0,0.0,0.0,1191.0,7.884019e+07
491,2019-12-13,0.0,0.0,0.0,0.0,1191.0,8.263966e+07
492,2019-12-16,0.0,0.0,0.0,0.0,1191.0,9.023861e+07
493,2019-12-17,0.0,0.0,0.0,0.0,1191.0,8.596420e+07


In [72]:
df_actions_and_account_value_ddpg.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModel/Fraksi3/df_actions_and_account_value_ddpg(4).csv',index=False)

##7.4 Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi (dalam bentuk string)

###7.4.1. Agen 1: A2C

In [73]:
#Replace NaN value to 0
action_columns_a2c=action_columns_a2c.fillna(0)

list_actions_str = []

numberOfRows = len(action_columns_a2c)

j=0
for j in range(numberOfRows):
  for i in action_columns_a2c:
   val = action_columns_a2c[i].values[j]
   #print(val)
   val_str = val.astype(str)
   if val > 0:
    val_str = "Beli " + val_str + " saham"
   elif val == 0:
     val_str = "Tahan"
   else:
     val_str = "Jual " + val_str + " saham"
   list_actions_str.append(val_str)
  j+=1

arr_actions_str = np.array(list_actions_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

add_column = pd.DataFrame(df_actions_and_account_value_a2c, columns = ['date','total_shares','account_value'])
df_actions_str = pd.concat([add_column, df_actions_str], axis=1)

indexNumberOfColumns_dfActionsStr = len(df_actions_str.columns)-1
  
# shift column 'total_shares' to last position
total_shares_column = df_actions_str.pop('total_shares')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'total_shares', total_shares_column)

# shift column 'account_value' to last position
account_value_column = df_actions_str.pop('account_value')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'account_value', account_value_column)

df_actions_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModelString/Fraksi3/df_actions_and_account_str_value_a2c(4).csv',index=False)
df_actions_str

,date,AKRA.JK,BRPT.JK,KLBF.JK,MEDC.JK,total_shares,account_value
0,2018-01-02,Tahan,Tahan,Beli 46.0 saham,Beli 23.0 saham,69.0,1.000000e+08
1,2018-01-03,Tahan,Tahan,Beli 46.0 saham,Beli 23.0 saham,138.0,9.982446e+07
2,2018-01-04,Tahan,Tahan,Beli 46.0 saham,Beli 23.0 saham,207.0,1.002399e+08
3,2018-01-05,Tahan,Tahan,Beli 46.0 saham,Beli 23.0 saham,276.0,1.007098e+08
4,2018-01-08,Tahan,Tahan,Beli 46.0 saham,Beli 23.0 saham,345.0,1.012161e+08
...,...,...,...,...,...,...,...
490,2019-12-12,Tahan,Tahan,Tahan,Tahan,1004.0,6.722608e+07
491,2019-12-13,Tahan,Tahan,Tahan,Tahan,1004.0,6.895085e+07
492,2019-12-16,Tahan,Tahan,Tahan,Tahan,1004.0,7.090821e+07
493,2019-12-17,Tahan,Tahan,Tahan,Tahan,1004.0,7.003610e+07


###7.4.2. Agen 2: PPO

In [74]:
#Replace NaN value to 0
action_columns_ppo=action_columns_ppo.fillna(0)

list_actions_str = []

numberOfRows = len(action_columns_ppo)

j=0
for j in range(numberOfRows):
  for i in action_columns_ppo:
   val = action_columns_ppo[i].values[j]
   #print(val)
   val_str = val.astype(str)
   if val > 0:
    val_str = "Beli " + val_str + " saham"
   elif val == 0:
     val_str = "Tahan"
   else:
     val_str = "Jual " + val_str + " saham"
   list_actions_str.append(val_str)
  j+=1

arr_actions_str = np.array(list_actions_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

add_column = pd.DataFrame(df_actions_and_account_value_ppo, columns = ['date','total_shares','account_value'])
df_actions_str = pd.concat([add_column, df_actions_str], axis=1)

indexNumberOfColumns_dfActionsStr = len(df_actions_str.columns)-1
  
# shift column 'total_shares' to last position
total_shares_column = df_actions_str.pop('total_shares')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'total_shares', total_shares_column)

# shift column 'account_value' to last position
account_value_column = df_actions_str.pop('account_value')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'account_value', account_value_column)

df_actions_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModelString/Fraksi3/df_actions_and_account_str_value_ppo(4).csv',index=False)
df_actions_str

,date,AKRA.JK,BRPT.JK,KLBF.JK,MEDC.JK,total_shares,account_value
0,2018-01-02,Tahan,Beli 1.0 saham,Beli 19.0 saham,Tahan,20.0,1.000000e+08
1,2018-01-03,Tahan,Beli 1.0 saham,Beli 19.0 saham,Tahan,40.0,9.993060e+07
2,2018-01-04,Tahan,Beli 1.0 saham,Beli 19.0 saham,Tahan,60.0,1.000926e+08
3,2018-01-05,Tahan,Beli 1.0 saham,Beli 19.0 saham,Tahan,80.0,1.001660e+08
4,2018-01-08,Tahan,Beli 1.0 saham,Beli 19.0 saham,Tahan,100.0,1.003305e+08
...,...,...,...,...,...,...,...
490,2019-12-12,Tahan,Tahan,Tahan,Tahan,674.0,9.950485e+07
491,2019-12-13,Tahan,Tahan,Tahan,Tahan,674.0,1.013487e+08
492,2019-12-16,Tahan,Tahan,Tahan,Tahan,674.0,9.978965e+07
493,2019-12-17,Tahan,Tahan,Tahan,Tahan,674.0,1.035111e+08


###7.4.1. Agen 3: TD3

In [75]:
#Replace NaN value to 0
action_columns_td3=action_columns_td3.fillna(0)

list_actions_str = []

numberOfRows = len(action_columns_td3)

j=0
for j in range(numberOfRows):
  for i in action_columns_td3:
   val = action_columns_td3[i].values[j]
   #print(val)
   val_str = val.astype(str)
   if val > 0:
    val_str = "Beli " + val_str + " saham"
   elif val == 0:
     val_str = "Tahan"
   else:
     val_str = "Jual " + val_str + " saham"
   list_actions_str.append(val_str)
  j+=1

arr_actions_str = np.array(list_actions_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

add_column = pd.DataFrame(df_actions_and_account_value_td3, columns = ['date','total_shares','account_value'])
df_actions_str = pd.concat([add_column, df_actions_str], axis=1)

indexNumberOfColumns_dfActionsStr = len(df_actions_str.columns)-1
  
# shift column 'total_shares' to last position
total_shares_column = df_actions_str.pop('total_shares')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'total_shares', total_shares_column)

# shift column 'account_value' to last position
account_value_column = df_actions_str.pop('account_value')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'account_value', account_value_column)

df_actions_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModelString/Fraksi3/df_actions_and_account_str_value_td3(4).csv',index=False)
df_actions_str

,date,AKRA.JK,BRPT.JK,KLBF.JK,MEDC.JK,total_shares,account_value
0,2018-01-02,Tahan,Beli 100.0 saham,Tahan,Beli 100.0 saham,200.0,1.000000e+08
1,2018-01-03,Tahan,Beli 100.0 saham,Tahan,Beli 100.0 saham,400.0,9.971899e+07
2,2018-01-04,Tahan,Beli 100.0 saham,Tahan,Beli 100.0 saham,600.0,9.942161e+07
3,2018-01-05,Tahan,Beli 100.0 saham,Tahan,Beli 100.0 saham,800.0,1.010046e+08
4,2018-01-08,Tahan,Beli 100.0 saham,Tahan,Beli 100.0 saham,1000.0,1.013782e+08
...,...,...,...,...,...,...,...
490,2019-12-12,Tahan,Tahan,Tahan,Tahan,1656.0,1.687853e+08
491,2019-12-13,Tahan,Tahan,Tahan,Tahan,1656.0,1.723084e+08
492,2019-12-16,Tahan,Tahan,Tahan,Tahan,1656.0,1.765704e+08
493,2019-12-17,Tahan,Tahan,Tahan,Tahan,1656.0,1.758882e+08


###7.4.1. Agen 4: SAC

In [76]:
#Replace NaN value to 0
action_columns_sac=action_columns_sac.fillna(0)

list_actions_str = []

numberOfRows = len(action_columns_sac)

j=0
for j in range(numberOfRows):
  for i in action_columns_sac:
   val = action_columns_sac[i].values[j]
   #print(val)
   val_str = val.astype(str)
   if val > 0:
    val_str = "Beli " + val_str + " saham"
   elif val == 0:
     val_str = "Tahan"
   else:
     val_str = "Jual " + val_str + " saham"
   list_actions_str.append(val_str)
  j+=1

arr_actions_str = np.array(list_actions_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

add_column = pd.DataFrame(df_actions_and_account_value_sac, columns = ['date','total_shares','account_value'])
df_actions_str = pd.concat([add_column, df_actions_str], axis=1)

indexNumberOfColumns_dfActionsStr = len(df_actions_str.columns)-1
  
# shift column 'total_shares' to last position
total_shares_column = df_actions_str.pop('total_shares')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'total_shares', total_shares_column)

# shift column 'account_value' to last position
account_value_column = df_actions_str.pop('account_value')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'account_value', account_value_column)

df_actions_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModelString/Fraksi3/df_actions_and_account_str_value_sac(4).csv',index=False)
df_actions_str

,date,AKRA.JK,BRPT.JK,KLBF.JK,MEDC.JK,total_shares,account_value
0,2018-01-02,Beli 100.0 saham,Beli 100.0 saham,Beli 100.0 saham,Tahan,300.0,1.000000e+08
1,2018-01-03,Beli 100.0 saham,Beli 100.0 saham,Beli 100.0 saham,Tahan,600.0,9.935796e+07
2,2018-01-04,Beli 100.0 saham,Beli 100.0 saham,Beli 100.0 saham,Tahan,900.0,9.992683e+07
3,2018-01-05,Tahan,Beli 1.0 saham,Beli 63.0 saham,Tahan,964.0,1.005588e+08
4,2018-01-08,Tahan,Tahan,Tahan,Tahan,964.0,1.025196e+08
...,...,...,...,...,...,...,...
490,2019-12-12,Tahan,Tahan,Tahan,Tahan,964.0,1.160527e+08
491,2019-12-13,Tahan,Tahan,Tahan,Tahan,964.0,1.177527e+08
492,2019-12-16,Tahan,Tahan,Tahan,Tahan,964.0,1.166092e+08
493,2019-12-17,Tahan,Tahan,Tahan,Tahan,964.0,1.195592e+08


###7.4.1. Agen 5: DDPG

In [77]:
#Replace NaN value to 0
action_columns_ddpg=action_columns_ddpg.fillna(0)

list_actions_str = []

numberOfRows = len(action_columns_ddpg)

j=0
for j in range(numberOfRows):
  for i in action_columns_ddpg:
   val = action_columns_ddpg[i].values[j]
   #print(val)
   val_str = val.astype(str)
   if val > 0:
    val_str = "Beli " + val_str + " saham"
   elif val == 0:
     val_str = "Tahan"
   else:
     val_str = "Jual " + val_str + " saham"
   list_actions_str.append(val_str)
  j+=1

arr_actions_str = np.array(list_actions_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

add_column = pd.DataFrame(df_actions_and_account_value_ddpg, columns = ['date','total_shares','account_value'])
df_actions_str = pd.concat([add_column, df_actions_str], axis=1)

indexNumberOfColumns_dfActionsStr = len(df_actions_str.columns)-1
  
# shift column 'total_shares' to last position
total_shares_column = df_actions_str.pop('total_shares')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'total_shares', total_shares_column)

# shift column 'account_value' to last position
account_value_column = df_actions_str.pop('account_value')

# insert column using insert(position,column_name, total_shares_column) function
df_actions_str.insert(indexNumberOfColumns_dfActionsStr, 'account_value', account_value_column)

df_actions_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/ActionsPerModelString/Fraksi3/df_actions_and_account_str_value_ddpg(4).csv',index=False)
df_actions_str

,date,AKRA.JK,BRPT.JK,KLBF.JK,MEDC.JK,total_shares,account_value
0,2018-01-02,Tahan,Beli 100.0 saham,Tahan,Beli 100.0 saham,200.0,1.000000e+08
1,2018-01-03,Tahan,Beli 100.0 saham,Tahan,Beli 100.0 saham,400.0,9.971899e+07
2,2018-01-04,Tahan,Beli 100.0 saham,Tahan,Beli 100.0 saham,600.0,9.942161e+07
3,2018-01-05,Tahan,Beli 100.0 saham,Tahan,Beli 100.0 saham,800.0,1.010046e+08
4,2018-01-08,Tahan,Beli 100.0 saham,Tahan,Beli 100.0 saham,1000.0,1.013782e+08
...,...,...,...,...,...,...,...
490,2019-12-12,Tahan,Tahan,Tahan,Tahan,1191.0,7.884019e+07
491,2019-12-13,Tahan,Tahan,Tahan,Tahan,1191.0,8.263966e+07
492,2019-12-16,Tahan,Tahan,Tahan,Tahan,1191.0,9.023861e+07
493,2019-12-17,Tahan,Tahan,Tahan,Tahan,1191.0,8.596420e+07


<a id='6.4'></a>
##7.5. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Kondisi dan Aksi

<a id='6.4.1'></a>
###7.5.1. Agen 1: A2C

In [78]:
#drop some columns
df_actions_and_account_value_a2c = df_actions_and_account_value_a2c.drop(['date', 'total_shares', 'account_value'], axis=1)

# create empty list
dataframes_list = []

#read the dataset of each stock, then put them on dataframes_list
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  mask = (df_temp['date'] >= TRADE_START_DATE) & (df_temp['date'] <= TRADE_END_DATE)
  df_temp = df_temp.loc[mask]
  dataframes_list.append(df_temp)
  #display(df_temp)

#separate each column of trading dataframe action, then put them on action_per_tic_list
action_per_tic_list = []
for i in df_actions_and_account_value_a2c:
    df_temp = df_actions_and_account_value_a2c[[i]]
    action_per_tic_list.append(df_temp)
    #display(df_temp)

# merging each dataframes_list members and each action_per_tic_list members
j=0
for i in dataframes_list:
  if j<numberOfTic-1:
   i['action'] = action_per_tic_list[j].iloc[:, 0].values.tolist()
   display(i)
   i.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActions/Fraksi3/1_A2C/df_states_and_actions_a2c_'+list_ticker[j]+'.csv',index=False)
   j+=1

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13136,2018-01-02,AKRA.JK,1275.0,1285.0,1235.0,1008.418701,25448000.0,1.0,-12.741437,42.797321,-27.402704,9.77,4.977856,0.0
13140,2018-01-03,AKRA.JK,1260.0,1265.0,1230.0,1004.352600,34013000.0,2.0,-13.495459,42.414932,-48.046638,9.15,6.195140,0.0
13144,2018-01-04,AKRA.JK,1240.0,1250.0,1240.0,1008.418701,21831000.0,3.0,-13.608060,42.942315,-44.180827,9.22,7.896003,0.0
13148,2018-01-05,AKRA.JK,1240.0,1240.0,1230.0,1008.418701,22779000.0,4.0,-13.541203,42.942315,-60.188004,9.22,2.734137,0.0
13160,2018-01-08,AKRA.JK,1250.0,1290.0,1245.0,1049.080688,44097500.0,0.0,-10.090815,48.035401,71.309493,9.52,5.385585,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,AKRA.JK,754.0,756.0,740.0,672.013977,36956500.0,3.0,-1.634581,49.494913,40.825594,13.94,0.220926,0.0
15976,2019-12-13,AKRA.JK,758.0,770.0,754.0,684.491699,58397500.0,4.0,0.871302,51.475430,75.206993,12.63,4.513726,0.0
15988,2019-12-16,AKRA.JK,768.0,778.0,756.0,689.839233,62960000.0,0.0,3.251256,52.304633,89.005454,12.14,14.500617,0.0
15992,2019-12-17,AKRA.JK,774.0,778.0,766.0,689.839233,53565500.0,1.0,5.078840,52.304633,97.518591,12.29,9.745929,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13137,2018-01-02,BRPT.JK,454.0,454.0,440.0,438.999084,110809000.0,1.0,1.533834,53.320545,-60.693822,9.77,4.977856,0.0
13141,2018-01-03,BRPT.JK,450.0,450.0,424.0,417.536926,133037000.0,2.0,-0.774940,47.187261,-138.640370,9.15,6.195140,0.0
13145,2018-01-04,BRPT.JK,428.0,430.0,406.0,399.976959,206900500.0,3.0,-3.975773,43.000787,-226.780331,9.22,7.896003,0.0
13149,2018-01-05,BRPT.JK,410.0,422.0,410.0,409.732513,156436000.0,4.0,-5.660019,45.766110,-203.154314,9.22,2.734137,0.0
13161,2018-01-08,BRPT.JK,420.0,426.0,412.0,407.781372,165324500.0,0.0,-7.070730,45.311286,-184.295229,9.52,5.385585,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,BRPT.JK,1430.0,1435.0,1405.0,1400.250610,58912700.0,3.0,95.708893,68.859335,92.321350,13.94,0.220926,0.0
15977,2019-12-13,BRPT.JK,1415.0,1420.0,1380.0,1410.181396,75925900.0,4.0,93.046575,69.276538,83.889991,12.63,4.513726,0.0
15989,2019-12-16,BRPT.JK,1410.0,1430.0,1405.0,1395.285156,74485300.0,0.0,88.712049,67.865673,81.687791,12.14,14.500617,0.0
15993,2019-12-17,BRPT.JK,1405.0,1435.0,1380.0,1425.077637,72110900.0,1.0,86.681698,69.164937,78.838577,12.29,9.745929,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13138,2018-01-02,KLBF.JK,1690.0,1730.0,1675.0,1568.763916,17137900.0,1.0,13.548843,58.402810,189.233758,9.77,4.977856,46.0
13142,2018-01-03,KLBF.JK,1730.0,1730.0,1675.0,1537.025879,15571500.0,2.0,13.566010,53.885335,138.052325,9.15,6.195140,46.0
13146,2018-01-04,KLBF.JK,1695.0,1745.0,1685.0,1582.365845,17727400.0,3.0,17.041725,58.615967,190.347106,9.22,7.896003,46.0
13150,2018-01-05,KLBF.JK,1730.0,1760.0,1730.0,1595.967896,16178100.0,4.0,20.655719,59.892843,227.496530,9.22,2.734137,46.0
13162,2018-01-08,KLBF.JK,1770.0,1785.0,1740.0,1618.637939,12831400.0,0.0,25.060240,61.918655,237.953502,9.52,5.385585,46.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,KLBF.JK,1550.0,1590.0,1545.0,1480.120605,26446200.0,3.0,-0.613375,49.672440,42.118931,13.94,0.220926,0.0
15978,2019-12-13,KLBF.JK,1600.0,1600.0,1575.0,1508.403076,18588700.0,4.0,2.470021,52.781494,100.013048,12.63,4.513726,0.0
15990,2019-12-16,KLBF.JK,1605.0,1610.0,1565.0,1484.834351,16279800.0,0.0,2.977512,50.112717,80.762455,12.14,14.500617,0.0
15994,2019-12-17,KLBF.JK,1585.0,1635.0,1555.0,1541.399414,46622200.0,1.0,7.853500,55.676841,134.244489,12.29,9.745929,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13139,2018-01-02,MEDC.JK,753.439148,770.370361,753.439148,713.797363,45424968.0,1.0,15.726914,60.442577,79.238661,9.77,4.977856,23.0
13143,2018-01-03,MEDC.JK,757.671936,770.370361,753.439148,709.809631,46648035.0,2.0,14.917566,59.654790,69.101356,9.15,6.195140,23.0
13147,2018-01-04,MEDC.JK,770.370361,770.370361,757.671936,713.797363,49596080.0,3.0,14.431571,60.191531,74.229441,9.22,7.896003,23.0
13151,2018-01-05,MEDC.JK,766.137573,812.698425,761.904785,757.661987,118722003.0,4.0,17.385515,65.425597,175.835426,9.22,2.734137,23.0
13163,2018-01-08,MEDC.JK,829.629639,838.095215,804.232788,769.625061,97001533.0,0.0,20.456053,66.662260,250.291888,9.52,5.385585,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,MEDC.JK,698.412720,715.343933,681.481506,661.957336,87890315.0,3.0,28.039621,66.014572,205.825025,13.94,0.220926,0.0
15979,2019-12-13,MEDC.JK,715.343933,744.973572,711.111084,693.858887,113998066.0,4.0,33.547092,69.154002,219.392372,12.63,4.513726,0.0
15991,2019-12-16,MEDC.JK,740.740723,804.232788,736.507935,757.661987,225414866.0,0.0,42.569465,74.103417,248.805834,12.14,14.500617,0.0
15995,2019-12-17,MEDC.JK,800.000000,833.862427,757.671936,721.772766,169035575.0,1.0,46.290195,67.775334,217.871857,12.29,9.745929,0.0


####String Action Version

In [79]:
numberOfColumns = len(dataframes_list[0].columns)
numberOfRows = len(dataframes_list[0])

list_actionCell_str=[]

j=0
for j in range(len(i)):
  for i in dataframes_list:
    actionCell= i["action"].values[j]
    actionCell_str = actionCell.astype(str)
    if actionCell > 0:
      actionCell_str = "Beli " + actionCell_str + " saham"
    elif val == 0:
      actionCell_str = "Tahan"
    else:
      actionCell_str = "Jual " + actionCell_str + " saham"
    list_actionCell_str.append(actionCell_str)
  j+=1

arr_actions_str = np.array(list_actionCell_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

j=0
for i in dataframes_list:
  #drop a column
  #i = i.drop(['action','tic'], axis=1)

  #reset index
  i=i.reset_index()

  #selecting column of actions in every tics  
  action_columns_str = df_actions_str.iloc[:,j]
  
  df_action_columns_str = pd.concat([i, action_columns_str], axis=1)
  df_action_columns_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActionsString/Fraksi3/1_A2C/df_states_and_actions_str_a2c_'+list_ticker[j]+'.csv',index=False)
  display(df_action_columns_str)
  j+=1

,index,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action,AKRA.JK
0,13136,2018-01-02,AKRA.JK,1275.0,1285.0,1235.0,1008.418701,25448000.0,1.0,-12.741437,42.797321,-27.402704,9.77,4.977856,0.0,Tahan
1,13140,2018-01-03,AKRA.JK,1260.0,1265.0,1230.0,1004.352600,34013000.0,2.0,-13.495459,42.414932,-48.046638,9.15,6.195140,0.0,Tahan
2,13144,2018-01-04,AKRA.JK,1240.0,1250.0,1240.0,1008.418701,21831000.0,3.0,-13.608060,42.942315,-44.180827,9.22,7.896003,0.0,Tahan
3,13148,2018-01-05,AKRA.JK,1240.0,1240.0,1230.0,1008.418701,22779000.0,4.0,-13.541203,42.942315,-60.188004,9.22,2.734137,0.0,Tahan
4,13160,2018-01-08,AKRA.JK,1250.0,1290.0,1245.0,1049.080688,44097500.0,0.0,-10.090815,48.035401,71.309493,9.52,5.385585,0.0,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15972,2019-12-12,AKRA.JK,754.0,756.0,740.0,672.013977,36956500.0,3.0,-1.634581,49.494913,40.825594,13.94,0.220926,0.0,Tahan
491,15976,2019-12-13,AKRA.JK,758.0,770.0,754.0,684.491699,58397500.0,4.0,0.871302,51.475430,75.206993,12.63,4.513726,0.0,Tahan
492,15988,2019-12-16,AKRA.JK,768.0,778.0,756.0,689.839233,62960000.0,0.0,3.251256,52.304633,89.005454,12.14,14.500617,0.0,Tahan
493,15992,2019-12-17,AKRA.JK,774.0,778.0,766.0,689.839233,53565500.0,1.0,5.078840,52.304633,97.518591,12.29,9.745929,0.0,Tahan


,index,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action,BRPT.JK
0,13137,2018-01-02,BRPT.JK,454.0,454.0,440.0,438.999084,110809000.0,1.0,1.533834,53.320545,-60.693822,9.77,4.977856,0.0,Tahan
1,13141,2018-01-03,BRPT.JK,450.0,450.0,424.0,417.536926,133037000.0,2.0,-0.774940,47.187261,-138.640370,9.15,6.195140,0.0,Tahan
2,13145,2018-01-04,BRPT.JK,428.0,430.0,406.0,399.976959,206900500.0,3.0,-3.975773,43.000787,-226.780331,9.22,7.896003,0.0,Tahan
3,13149,2018-01-05,BRPT.JK,410.0,422.0,410.0,409.732513,156436000.0,4.0,-5.660019,45.766110,-203.154314,9.22,2.734137,0.0,Tahan
4,13161,2018-01-08,BRPT.JK,420.0,426.0,412.0,407.781372,165324500.0,0.0,-7.070730,45.311286,-184.295229,9.52,5.385585,0.0,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15973,2019-12-12,BRPT.JK,1430.0,1435.0,1405.0,1400.250610,58912700.0,3.0,95.708893,68.859335,92.321350,13.94,0.220926,0.0,Tahan
491,15977,2019-12-13,BRPT.JK,1415.0,1420.0,1380.0,1410.181396,75925900.0,4.0,93.046575,69.276538,83.889991,12.63,4.513726,0.0,Tahan
492,15989,2019-12-16,BRPT.JK,1410.0,1430.0,1405.0,1395.285156,74485300.0,0.0,88.712049,67.865673,81.687791,12.14,14.500617,0.0,Tahan
493,15993,2019-12-17,BRPT.JK,1405.0,1435.0,1380.0,1425.077637,72110900.0,1.0,86.681698,69.164937,78.838577,12.29,9.745929,0.0,Tahan


,index,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action,KLBF.JK
0,13138,2018-01-02,KLBF.JK,1690.0,1730.0,1675.0,1568.763916,17137900.0,1.0,13.548843,58.402810,189.233758,9.77,4.977856,46.0,Beli 46.0 saham
1,13142,2018-01-03,KLBF.JK,1730.0,1730.0,1675.0,1537.025879,15571500.0,2.0,13.566010,53.885335,138.052325,9.15,6.195140,46.0,Beli 46.0 saham
2,13146,2018-01-04,KLBF.JK,1695.0,1745.0,1685.0,1582.365845,17727400.0,3.0,17.041725,58.615967,190.347106,9.22,7.896003,46.0,Beli 46.0 saham
3,13150,2018-01-05,KLBF.JK,1730.0,1760.0,1730.0,1595.967896,16178100.0,4.0,20.655719,59.892843,227.496530,9.22,2.734137,46.0,Beli 46.0 saham
4,13162,2018-01-08,KLBF.JK,1770.0,1785.0,1740.0,1618.637939,12831400.0,0.0,25.060240,61.918655,237.953502,9.52,5.385585,46.0,Beli 46.0 saham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15974,2019-12-12,KLBF.JK,1550.0,1590.0,1545.0,1480.120605,26446200.0,3.0,-0.613375,49.672440,42.118931,13.94,0.220926,0.0,Tahan
491,15978,2019-12-13,KLBF.JK,1600.0,1600.0,1575.0,1508.403076,18588700.0,4.0,2.470021,52.781494,100.013048,12.63,4.513726,0.0,Tahan
492,15990,2019-12-16,KLBF.JK,1605.0,1610.0,1565.0,1484.834351,16279800.0,0.0,2.977512,50.112717,80.762455,12.14,14.500617,0.0,Tahan
493,15994,2019-12-17,KLBF.JK,1585.0,1635.0,1555.0,1541.399414,46622200.0,1.0,7.853500,55.676841,134.244489,12.29,9.745929,0.0,Tahan


,index,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action,MEDC.JK
0,13139,2018-01-02,MEDC.JK,753.439148,770.370361,753.439148,713.797363,45424968.0,1.0,15.726914,60.442577,79.238661,9.77,4.977856,23.0,Beli 23.0 saham
1,13143,2018-01-03,MEDC.JK,757.671936,770.370361,753.439148,709.809631,46648035.0,2.0,14.917566,59.654790,69.101356,9.15,6.195140,23.0,Beli 23.0 saham
2,13147,2018-01-04,MEDC.JK,770.370361,770.370361,757.671936,713.797363,49596080.0,3.0,14.431571,60.191531,74.229441,9.22,7.896003,23.0,Beli 23.0 saham
3,13151,2018-01-05,MEDC.JK,766.137573,812.698425,761.904785,757.661987,118722003.0,4.0,17.385515,65.425597,175.835426,9.22,2.734137,23.0,Beli 23.0 saham
4,13163,2018-01-08,MEDC.JK,829.629639,838.095215,804.232788,769.625061,97001533.0,0.0,20.456053,66.662260,250.291888,9.52,5.385585,23.0,Beli 23.0 saham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15975,2019-12-12,MEDC.JK,698.412720,715.343933,681.481506,661.957336,87890315.0,3.0,28.039621,66.014572,205.825025,13.94,0.220926,0.0,Tahan
491,15979,2019-12-13,MEDC.JK,715.343933,744.973572,711.111084,693.858887,113998066.0,4.0,33.547092,69.154002,219.392372,12.63,4.513726,0.0,Tahan
492,15991,2019-12-16,MEDC.JK,740.740723,804.232788,736.507935,757.661987,225414866.0,0.0,42.569465,74.103417,248.805834,12.14,14.500617,0.0,Tahan
493,15995,2019-12-17,MEDC.JK,800.000000,833.862427,757.671936,721.772766,169035575.0,1.0,46.290195,67.775334,217.871857,12.29,9.745929,0.0,Tahan


<a id='6.4.2'></a>
###7.5.2.Agen 2: PPO

In [80]:
#drop some columns
df_actions_and_account_value_ppo = df_actions_and_account_value_ppo.drop(['date', 'total_shares', 'account_value'], axis=1)

# create empty list
dataframes_list = []

#read the dataset of each stock, then put them on dataframes_list
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  mask = (df_temp['date'] >= TRADE_START_DATE) & (df_temp['date'] <= TRADE_END_DATE)
  df_temp = df_temp.loc[mask]
  dataframes_list.append(df_temp)
  #display(df_temp)

#separate each column of trading dataframe action, then put them on action_per_tic_list
action_per_tic_list = []
for i in df_actions_and_account_value_ppo:
    df_temp = df_actions_and_account_value_ppo[[i]]
    action_per_tic_list.append(df_temp)
    #display(df_temp)

# merging each dataframes_list members and each action_per_tic_list members
j=0
for i in dataframes_list:
  if j<numberOfTic:
   i['action'] = action_per_tic_list[j].iloc[:, 0].values.tolist()
   display(i)
   i.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActions/Fraksi3/2_PPO/df_states_and_actions_ppo_'+list_ticker[j]+'.csv',index=False)
   j+=1

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13136,2018-01-02,AKRA.JK,1275.0,1285.0,1235.0,1008.418701,25448000.0,1.0,-12.741437,42.797321,-27.402704,9.77,4.977856,0.0
13140,2018-01-03,AKRA.JK,1260.0,1265.0,1230.0,1004.352600,34013000.0,2.0,-13.495459,42.414932,-48.046638,9.15,6.195140,0.0
13144,2018-01-04,AKRA.JK,1240.0,1250.0,1240.0,1008.418701,21831000.0,3.0,-13.608060,42.942315,-44.180827,9.22,7.896003,0.0
13148,2018-01-05,AKRA.JK,1240.0,1240.0,1230.0,1008.418701,22779000.0,4.0,-13.541203,42.942315,-60.188004,9.22,2.734137,0.0
13160,2018-01-08,AKRA.JK,1250.0,1290.0,1245.0,1049.080688,44097500.0,0.0,-10.090815,48.035401,71.309493,9.52,5.385585,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,AKRA.JK,754.0,756.0,740.0,672.013977,36956500.0,3.0,-1.634581,49.494913,40.825594,13.94,0.220926,0.0
15976,2019-12-13,AKRA.JK,758.0,770.0,754.0,684.491699,58397500.0,4.0,0.871302,51.475430,75.206993,12.63,4.513726,0.0
15988,2019-12-16,AKRA.JK,768.0,778.0,756.0,689.839233,62960000.0,0.0,3.251256,52.304633,89.005454,12.14,14.500617,0.0
15992,2019-12-17,AKRA.JK,774.0,778.0,766.0,689.839233,53565500.0,1.0,5.078840,52.304633,97.518591,12.29,9.745929,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13137,2018-01-02,BRPT.JK,454.0,454.0,440.0,438.999084,110809000.0,1.0,1.533834,53.320545,-60.693822,9.77,4.977856,1.0
13141,2018-01-03,BRPT.JK,450.0,450.0,424.0,417.536926,133037000.0,2.0,-0.774940,47.187261,-138.640370,9.15,6.195140,1.0
13145,2018-01-04,BRPT.JK,428.0,430.0,406.0,399.976959,206900500.0,3.0,-3.975773,43.000787,-226.780331,9.22,7.896003,1.0
13149,2018-01-05,BRPT.JK,410.0,422.0,410.0,409.732513,156436000.0,4.0,-5.660019,45.766110,-203.154314,9.22,2.734137,1.0
13161,2018-01-08,BRPT.JK,420.0,426.0,412.0,407.781372,165324500.0,0.0,-7.070730,45.311286,-184.295229,9.52,5.385585,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,BRPT.JK,1430.0,1435.0,1405.0,1400.250610,58912700.0,3.0,95.708893,68.859335,92.321350,13.94,0.220926,0.0
15977,2019-12-13,BRPT.JK,1415.0,1420.0,1380.0,1410.181396,75925900.0,4.0,93.046575,69.276538,83.889991,12.63,4.513726,0.0
15989,2019-12-16,BRPT.JK,1410.0,1430.0,1405.0,1395.285156,74485300.0,0.0,88.712049,67.865673,81.687791,12.14,14.500617,0.0
15993,2019-12-17,BRPT.JK,1405.0,1435.0,1380.0,1425.077637,72110900.0,1.0,86.681698,69.164937,78.838577,12.29,9.745929,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13138,2018-01-02,KLBF.JK,1690.0,1730.0,1675.0,1568.763916,17137900.0,1.0,13.548843,58.402810,189.233758,9.77,4.977856,19.0
13142,2018-01-03,KLBF.JK,1730.0,1730.0,1675.0,1537.025879,15571500.0,2.0,13.566010,53.885335,138.052325,9.15,6.195140,19.0
13146,2018-01-04,KLBF.JK,1695.0,1745.0,1685.0,1582.365845,17727400.0,3.0,17.041725,58.615967,190.347106,9.22,7.896003,19.0
13150,2018-01-05,KLBF.JK,1730.0,1760.0,1730.0,1595.967896,16178100.0,4.0,20.655719,59.892843,227.496530,9.22,2.734137,19.0
13162,2018-01-08,KLBF.JK,1770.0,1785.0,1740.0,1618.637939,12831400.0,0.0,25.060240,61.918655,237.953502,9.52,5.385585,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,KLBF.JK,1550.0,1590.0,1545.0,1480.120605,26446200.0,3.0,-0.613375,49.672440,42.118931,13.94,0.220926,0.0
15978,2019-12-13,KLBF.JK,1600.0,1600.0,1575.0,1508.403076,18588700.0,4.0,2.470021,52.781494,100.013048,12.63,4.513726,0.0
15990,2019-12-16,KLBF.JK,1605.0,1610.0,1565.0,1484.834351,16279800.0,0.0,2.977512,50.112717,80.762455,12.14,14.500617,0.0
15994,2019-12-17,KLBF.JK,1585.0,1635.0,1555.0,1541.399414,46622200.0,1.0,7.853500,55.676841,134.244489,12.29,9.745929,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13139,2018-01-02,MEDC.JK,753.439148,770.370361,753.439148,713.797363,45424968.0,1.0,15.726914,60.442577,79.238661,9.77,4.977856,0.0
13143,2018-01-03,MEDC.JK,757.671936,770.370361,753.439148,709.809631,46648035.0,2.0,14.917566,59.654790,69.101356,9.15,6.195140,0.0
13147,2018-01-04,MEDC.JK,770.370361,770.370361,757.671936,713.797363,49596080.0,3.0,14.431571,60.191531,74.229441,9.22,7.896003,0.0
13151,2018-01-05,MEDC.JK,766.137573,812.698425,761.904785,757.661987,118722003.0,4.0,17.385515,65.425597,175.835426,9.22,2.734137,0.0
13163,2018-01-08,MEDC.JK,829.629639,838.095215,804.232788,769.625061,97001533.0,0.0,20.456053,66.662260,250.291888,9.52,5.385585,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,MEDC.JK,698.412720,715.343933,681.481506,661.957336,87890315.0,3.0,28.039621,66.014572,205.825025,13.94,0.220926,0.0
15979,2019-12-13,MEDC.JK,715.343933,744.973572,711.111084,693.858887,113998066.0,4.0,33.547092,69.154002,219.392372,12.63,4.513726,0.0
15991,2019-12-16,MEDC.JK,740.740723,804.232788,736.507935,757.661987,225414866.0,0.0,42.569465,74.103417,248.805834,12.14,14.500617,0.0
15995,2019-12-17,MEDC.JK,800.000000,833.862427,757.671936,721.772766,169035575.0,1.0,46.290195,67.775334,217.871857,12.29,9.745929,0.0


####String Action Version

In [81]:
numberOfColumns = len(dataframes_list[0].columns)
numberOfRows = len(dataframes_list[0])

list_actionCell_str=[]

j=0
for j in range(len(i)):
  for i in dataframes_list:
    actionCell= i["action"].values[j]
    actionCell_str = actionCell.astype(str)
    if actionCell > 0:
      actionCell_str = "Beli " + actionCell_str + " saham"
    elif val == 0:
      actionCell_str = "Tahan"
    else:
      actionCell_str = "Jual " + actionCell_str + " saham"
    list_actionCell_str.append(actionCell_str)
  j+=1

arr_actions_str = np.array(list_actionCell_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

j=0
for i in dataframes_list:
  #drop a column
  i = i.drop(['action','tic'], axis=1)

  #reset index
  i=i.reset_index()

  #selecting column of actions in every tics  
  action_columns_str = df_actions_str.iloc[:,j]
  
  df_action_columns_str = pd.concat([i, action_columns_str], axis=1)
  df_action_columns_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActionsString/Fraksi3/2_PPO/df_states_and_actions_str_ppo_'+list_ticker[j]+'.csv',index=False)
  display(df_action_columns_str)
  j+=1

,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,AKRA.JK
0,13136,2018-01-02,1275.0,1285.0,1235.0,1008.418701,25448000.0,1.0,-12.741437,42.797321,-27.402704,9.77,4.977856,Tahan
1,13140,2018-01-03,1260.0,1265.0,1230.0,1004.352600,34013000.0,2.0,-13.495459,42.414932,-48.046638,9.15,6.195140,Tahan
2,13144,2018-01-04,1240.0,1250.0,1240.0,1008.418701,21831000.0,3.0,-13.608060,42.942315,-44.180827,9.22,7.896003,Tahan
3,13148,2018-01-05,1240.0,1240.0,1230.0,1008.418701,22779000.0,4.0,-13.541203,42.942315,-60.188004,9.22,2.734137,Tahan
4,13160,2018-01-08,1250.0,1290.0,1245.0,1049.080688,44097500.0,0.0,-10.090815,48.035401,71.309493,9.52,5.385585,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15972,2019-12-12,754.0,756.0,740.0,672.013977,36956500.0,3.0,-1.634581,49.494913,40.825594,13.94,0.220926,Tahan
491,15976,2019-12-13,758.0,770.0,754.0,684.491699,58397500.0,4.0,0.871302,51.475430,75.206993,12.63,4.513726,Tahan
492,15988,2019-12-16,768.0,778.0,756.0,689.839233,62960000.0,0.0,3.251256,52.304633,89.005454,12.14,14.500617,Tahan
493,15992,2019-12-17,774.0,778.0,766.0,689.839233,53565500.0,1.0,5.078840,52.304633,97.518591,12.29,9.745929,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,BRPT.JK
0,13137,2018-01-02,454.0,454.0,440.0,438.999084,110809000.0,1.0,1.533834,53.320545,-60.693822,9.77,4.977856,Beli 1.0 saham
1,13141,2018-01-03,450.0,450.0,424.0,417.536926,133037000.0,2.0,-0.774940,47.187261,-138.640370,9.15,6.195140,Beli 1.0 saham
2,13145,2018-01-04,428.0,430.0,406.0,399.976959,206900500.0,3.0,-3.975773,43.000787,-226.780331,9.22,7.896003,Beli 1.0 saham
3,13149,2018-01-05,410.0,422.0,410.0,409.732513,156436000.0,4.0,-5.660019,45.766110,-203.154314,9.22,2.734137,Beli 1.0 saham
4,13161,2018-01-08,420.0,426.0,412.0,407.781372,165324500.0,0.0,-7.070730,45.311286,-184.295229,9.52,5.385585,Beli 1.0 saham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15973,2019-12-12,1430.0,1435.0,1405.0,1400.250610,58912700.0,3.0,95.708893,68.859335,92.321350,13.94,0.220926,Tahan
491,15977,2019-12-13,1415.0,1420.0,1380.0,1410.181396,75925900.0,4.0,93.046575,69.276538,83.889991,12.63,4.513726,Tahan
492,15989,2019-12-16,1410.0,1430.0,1405.0,1395.285156,74485300.0,0.0,88.712049,67.865673,81.687791,12.14,14.500617,Tahan
493,15993,2019-12-17,1405.0,1435.0,1380.0,1425.077637,72110900.0,1.0,86.681698,69.164937,78.838577,12.29,9.745929,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,KLBF.JK
0,13138,2018-01-02,1690.0,1730.0,1675.0,1568.763916,17137900.0,1.0,13.548843,58.402810,189.233758,9.77,4.977856,Beli 19.0 saham
1,13142,2018-01-03,1730.0,1730.0,1675.0,1537.025879,15571500.0,2.0,13.566010,53.885335,138.052325,9.15,6.195140,Beli 19.0 saham
2,13146,2018-01-04,1695.0,1745.0,1685.0,1582.365845,17727400.0,3.0,17.041725,58.615967,190.347106,9.22,7.896003,Beli 19.0 saham
3,13150,2018-01-05,1730.0,1760.0,1730.0,1595.967896,16178100.0,4.0,20.655719,59.892843,227.496530,9.22,2.734137,Beli 19.0 saham
4,13162,2018-01-08,1770.0,1785.0,1740.0,1618.637939,12831400.0,0.0,25.060240,61.918655,237.953502,9.52,5.385585,Beli 19.0 saham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15974,2019-12-12,1550.0,1590.0,1545.0,1480.120605,26446200.0,3.0,-0.613375,49.672440,42.118931,13.94,0.220926,Tahan
491,15978,2019-12-13,1600.0,1600.0,1575.0,1508.403076,18588700.0,4.0,2.470021,52.781494,100.013048,12.63,4.513726,Tahan
492,15990,2019-12-16,1605.0,1610.0,1565.0,1484.834351,16279800.0,0.0,2.977512,50.112717,80.762455,12.14,14.500617,Tahan
493,15994,2019-12-17,1585.0,1635.0,1555.0,1541.399414,46622200.0,1.0,7.853500,55.676841,134.244489,12.29,9.745929,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,MEDC.JK
0,13139,2018-01-02,753.439148,770.370361,753.439148,713.797363,45424968.0,1.0,15.726914,60.442577,79.238661,9.77,4.977856,Tahan
1,13143,2018-01-03,757.671936,770.370361,753.439148,709.809631,46648035.0,2.0,14.917566,59.654790,69.101356,9.15,6.195140,Tahan
2,13147,2018-01-04,770.370361,770.370361,757.671936,713.797363,49596080.0,3.0,14.431571,60.191531,74.229441,9.22,7.896003,Tahan
3,13151,2018-01-05,766.137573,812.698425,761.904785,757.661987,118722003.0,4.0,17.385515,65.425597,175.835426,9.22,2.734137,Tahan
4,13163,2018-01-08,829.629639,838.095215,804.232788,769.625061,97001533.0,0.0,20.456053,66.662260,250.291888,9.52,5.385585,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15975,2019-12-12,698.412720,715.343933,681.481506,661.957336,87890315.0,3.0,28.039621,66.014572,205.825025,13.94,0.220926,Tahan
491,15979,2019-12-13,715.343933,744.973572,711.111084,693.858887,113998066.0,4.0,33.547092,69.154002,219.392372,12.63,4.513726,Tahan
492,15991,2019-12-16,740.740723,804.232788,736.507935,757.661987,225414866.0,0.0,42.569465,74.103417,248.805834,12.14,14.500617,Tahan
493,15995,2019-12-17,800.000000,833.862427,757.671936,721.772766,169035575.0,1.0,46.290195,67.775334,217.871857,12.29,9.745929,Tahan


<a id='6.4.3'></a>
###7.5.3. Agen 3: TD3

In [82]:
#drop some columns
df_actions_and_account_value_td3 = df_actions_and_account_value_td3.drop(['date', 'total_shares', 'account_value'], axis=1)

# create empty list
dataframes_list = []

#read the dataset of each stock, then put them on dataframes_list
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  mask = (df_temp['date'] >= TRADE_START_DATE) & (df_temp['date'] <= TRADE_END_DATE)
  df_temp = df_temp.loc[mask]
  dataframes_list.append(df_temp)
  #display(df_temp)

#separate each column of trading dataframe action, then put them on action_per_tic_list
action_per_tic_list = []
for i in df_actions_and_account_value_td3:
    df_temp = df_actions_and_account_value_td3[[i]]
    action_per_tic_list.append(df_temp)
    #display(df_temp)

# merging each dataframes_list members and each action_per_tic_list members
j=0
for i in dataframes_list:
  if j<numberOfTic:
   i['action'] = action_per_tic_list[j].iloc[:, 0].values.tolist()
   display(i)
   i.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActions/Fraksi3/3_TD3/df_states_and_actions_td3_'+list_ticker[j]+'.csv',index=False)
   j+=1

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13136,2018-01-02,AKRA.JK,1275.0,1285.0,1235.0,1008.418701,25448000.0,1.0,-12.741437,42.797321,-27.402704,9.77,4.977856,0.0
13140,2018-01-03,AKRA.JK,1260.0,1265.0,1230.0,1004.352600,34013000.0,2.0,-13.495459,42.414932,-48.046638,9.15,6.195140,0.0
13144,2018-01-04,AKRA.JK,1240.0,1250.0,1240.0,1008.418701,21831000.0,3.0,-13.608060,42.942315,-44.180827,9.22,7.896003,0.0
13148,2018-01-05,AKRA.JK,1240.0,1240.0,1230.0,1008.418701,22779000.0,4.0,-13.541203,42.942315,-60.188004,9.22,2.734137,0.0
13160,2018-01-08,AKRA.JK,1250.0,1290.0,1245.0,1049.080688,44097500.0,0.0,-10.090815,48.035401,71.309493,9.52,5.385585,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,AKRA.JK,754.0,756.0,740.0,672.013977,36956500.0,3.0,-1.634581,49.494913,40.825594,13.94,0.220926,0.0
15976,2019-12-13,AKRA.JK,758.0,770.0,754.0,684.491699,58397500.0,4.0,0.871302,51.475430,75.206993,12.63,4.513726,0.0
15988,2019-12-16,AKRA.JK,768.0,778.0,756.0,689.839233,62960000.0,0.0,3.251256,52.304633,89.005454,12.14,14.500617,0.0
15992,2019-12-17,AKRA.JK,774.0,778.0,766.0,689.839233,53565500.0,1.0,5.078840,52.304633,97.518591,12.29,9.745929,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13137,2018-01-02,BRPT.JK,454.0,454.0,440.0,438.999084,110809000.0,1.0,1.533834,53.320545,-60.693822,9.77,4.977856,100.0
13141,2018-01-03,BRPT.JK,450.0,450.0,424.0,417.536926,133037000.0,2.0,-0.774940,47.187261,-138.640370,9.15,6.195140,100.0
13145,2018-01-04,BRPT.JK,428.0,430.0,406.0,399.976959,206900500.0,3.0,-3.975773,43.000787,-226.780331,9.22,7.896003,100.0
13149,2018-01-05,BRPT.JK,410.0,422.0,410.0,409.732513,156436000.0,4.0,-5.660019,45.766110,-203.154314,9.22,2.734137,100.0
13161,2018-01-08,BRPT.JK,420.0,426.0,412.0,407.781372,165324500.0,0.0,-7.070730,45.311286,-184.295229,9.52,5.385585,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,BRPT.JK,1430.0,1435.0,1405.0,1400.250610,58912700.0,3.0,95.708893,68.859335,92.321350,13.94,0.220926,0.0
15977,2019-12-13,BRPT.JK,1415.0,1420.0,1380.0,1410.181396,75925900.0,4.0,93.046575,69.276538,83.889991,12.63,4.513726,0.0
15989,2019-12-16,BRPT.JK,1410.0,1430.0,1405.0,1395.285156,74485300.0,0.0,88.712049,67.865673,81.687791,12.14,14.500617,0.0
15993,2019-12-17,BRPT.JK,1405.0,1435.0,1380.0,1425.077637,72110900.0,1.0,86.681698,69.164937,78.838577,12.29,9.745929,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13138,2018-01-02,KLBF.JK,1690.0,1730.0,1675.0,1568.763916,17137900.0,1.0,13.548843,58.402810,189.233758,9.77,4.977856,0.0
13142,2018-01-03,KLBF.JK,1730.0,1730.0,1675.0,1537.025879,15571500.0,2.0,13.566010,53.885335,138.052325,9.15,6.195140,0.0
13146,2018-01-04,KLBF.JK,1695.0,1745.0,1685.0,1582.365845,17727400.0,3.0,17.041725,58.615967,190.347106,9.22,7.896003,0.0
13150,2018-01-05,KLBF.JK,1730.0,1760.0,1730.0,1595.967896,16178100.0,4.0,20.655719,59.892843,227.496530,9.22,2.734137,0.0
13162,2018-01-08,KLBF.JK,1770.0,1785.0,1740.0,1618.637939,12831400.0,0.0,25.060240,61.918655,237.953502,9.52,5.385585,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,KLBF.JK,1550.0,1590.0,1545.0,1480.120605,26446200.0,3.0,-0.613375,49.672440,42.118931,13.94,0.220926,0.0
15978,2019-12-13,KLBF.JK,1600.0,1600.0,1575.0,1508.403076,18588700.0,4.0,2.470021,52.781494,100.013048,12.63,4.513726,0.0
15990,2019-12-16,KLBF.JK,1605.0,1610.0,1565.0,1484.834351,16279800.0,0.0,2.977512,50.112717,80.762455,12.14,14.500617,0.0
15994,2019-12-17,KLBF.JK,1585.0,1635.0,1555.0,1541.399414,46622200.0,1.0,7.853500,55.676841,134.244489,12.29,9.745929,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13139,2018-01-02,MEDC.JK,753.439148,770.370361,753.439148,713.797363,45424968.0,1.0,15.726914,60.442577,79.238661,9.77,4.977856,100.0
13143,2018-01-03,MEDC.JK,757.671936,770.370361,753.439148,709.809631,46648035.0,2.0,14.917566,59.654790,69.101356,9.15,6.195140,100.0
13147,2018-01-04,MEDC.JK,770.370361,770.370361,757.671936,713.797363,49596080.0,3.0,14.431571,60.191531,74.229441,9.22,7.896003,100.0
13151,2018-01-05,MEDC.JK,766.137573,812.698425,761.904785,757.661987,118722003.0,4.0,17.385515,65.425597,175.835426,9.22,2.734137,100.0
13163,2018-01-08,MEDC.JK,829.629639,838.095215,804.232788,769.625061,97001533.0,0.0,20.456053,66.662260,250.291888,9.52,5.385585,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,MEDC.JK,698.412720,715.343933,681.481506,661.957336,87890315.0,3.0,28.039621,66.014572,205.825025,13.94,0.220926,0.0
15979,2019-12-13,MEDC.JK,715.343933,744.973572,711.111084,693.858887,113998066.0,4.0,33.547092,69.154002,219.392372,12.63,4.513726,0.0
15991,2019-12-16,MEDC.JK,740.740723,804.232788,736.507935,757.661987,225414866.0,0.0,42.569465,74.103417,248.805834,12.14,14.500617,0.0
15995,2019-12-17,MEDC.JK,800.000000,833.862427,757.671936,721.772766,169035575.0,1.0,46.290195,67.775334,217.871857,12.29,9.745929,0.0


####String Action Version

In [83]:
numberOfColumns = len(dataframes_list[0].columns)
numberOfRows = len(dataframes_list[0])

list_actionCell_str=[]

j=0
for j in range(len(i)):
  for i in dataframes_list:
    actionCell= i["action"].values[j]
    actionCell_str = actionCell.astype(str)
    if actionCell > 0:
      actionCell_str = "Beli " + actionCell_str + " saham"
    elif val == 0:
      actionCell_str = "Tahan"
    else:
      actionCell_str = "Jual " + actionCell_str + " saham"
    list_actionCell_str.append(actionCell_str)
  j+=1

arr_actions_str = np.array(list_actionCell_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

j=0
for i in dataframes_list:
  #drop a column
  i = i.drop(['action','tic'], axis=1)

  #reset index
  i=i.reset_index()

  #selecting column of actions in every tics  
  action_columns_str = df_actions_str.iloc[:,j]
  
  df_action_columns_str = pd.concat([i, action_columns_str], axis=1)
  df_action_columns_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActionsString/Fraksi3/3_TD3/df_states_and_actions_str_td3_'+list_ticker[j]+'.csv',index=False)
  display(df_action_columns_str)
  j+=1

,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,AKRA.JK
0,13136,2018-01-02,1275.0,1285.0,1235.0,1008.418701,25448000.0,1.0,-12.741437,42.797321,-27.402704,9.77,4.977856,Tahan
1,13140,2018-01-03,1260.0,1265.0,1230.0,1004.352600,34013000.0,2.0,-13.495459,42.414932,-48.046638,9.15,6.195140,Tahan
2,13144,2018-01-04,1240.0,1250.0,1240.0,1008.418701,21831000.0,3.0,-13.608060,42.942315,-44.180827,9.22,7.896003,Tahan
3,13148,2018-01-05,1240.0,1240.0,1230.0,1008.418701,22779000.0,4.0,-13.541203,42.942315,-60.188004,9.22,2.734137,Tahan
4,13160,2018-01-08,1250.0,1290.0,1245.0,1049.080688,44097500.0,0.0,-10.090815,48.035401,71.309493,9.52,5.385585,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15972,2019-12-12,754.0,756.0,740.0,672.013977,36956500.0,3.0,-1.634581,49.494913,40.825594,13.94,0.220926,Tahan
491,15976,2019-12-13,758.0,770.0,754.0,684.491699,58397500.0,4.0,0.871302,51.475430,75.206993,12.63,4.513726,Tahan
492,15988,2019-12-16,768.0,778.0,756.0,689.839233,62960000.0,0.0,3.251256,52.304633,89.005454,12.14,14.500617,Tahan
493,15992,2019-12-17,774.0,778.0,766.0,689.839233,53565500.0,1.0,5.078840,52.304633,97.518591,12.29,9.745929,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,BRPT.JK
0,13137,2018-01-02,454.0,454.0,440.0,438.999084,110809000.0,1.0,1.533834,53.320545,-60.693822,9.77,4.977856,Beli 100.0 saham
1,13141,2018-01-03,450.0,450.0,424.0,417.536926,133037000.0,2.0,-0.774940,47.187261,-138.640370,9.15,6.195140,Beli 100.0 saham
2,13145,2018-01-04,428.0,430.0,406.0,399.976959,206900500.0,3.0,-3.975773,43.000787,-226.780331,9.22,7.896003,Beli 100.0 saham
3,13149,2018-01-05,410.0,422.0,410.0,409.732513,156436000.0,4.0,-5.660019,45.766110,-203.154314,9.22,2.734137,Beli 100.0 saham
4,13161,2018-01-08,420.0,426.0,412.0,407.781372,165324500.0,0.0,-7.070730,45.311286,-184.295229,9.52,5.385585,Beli 100.0 saham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15973,2019-12-12,1430.0,1435.0,1405.0,1400.250610,58912700.0,3.0,95.708893,68.859335,92.321350,13.94,0.220926,Tahan
491,15977,2019-12-13,1415.0,1420.0,1380.0,1410.181396,75925900.0,4.0,93.046575,69.276538,83.889991,12.63,4.513726,Tahan
492,15989,2019-12-16,1410.0,1430.0,1405.0,1395.285156,74485300.0,0.0,88.712049,67.865673,81.687791,12.14,14.500617,Tahan
493,15993,2019-12-17,1405.0,1435.0,1380.0,1425.077637,72110900.0,1.0,86.681698,69.164937,78.838577,12.29,9.745929,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,KLBF.JK
0,13138,2018-01-02,1690.0,1730.0,1675.0,1568.763916,17137900.0,1.0,13.548843,58.402810,189.233758,9.77,4.977856,Tahan
1,13142,2018-01-03,1730.0,1730.0,1675.0,1537.025879,15571500.0,2.0,13.566010,53.885335,138.052325,9.15,6.195140,Tahan
2,13146,2018-01-04,1695.0,1745.0,1685.0,1582.365845,17727400.0,3.0,17.041725,58.615967,190.347106,9.22,7.896003,Tahan
3,13150,2018-01-05,1730.0,1760.0,1730.0,1595.967896,16178100.0,4.0,20.655719,59.892843,227.496530,9.22,2.734137,Tahan
4,13162,2018-01-08,1770.0,1785.0,1740.0,1618.637939,12831400.0,0.0,25.060240,61.918655,237.953502,9.52,5.385585,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15974,2019-12-12,1550.0,1590.0,1545.0,1480.120605,26446200.0,3.0,-0.613375,49.672440,42.118931,13.94,0.220926,Tahan
491,15978,2019-12-13,1600.0,1600.0,1575.0,1508.403076,18588700.0,4.0,2.470021,52.781494,100.013048,12.63,4.513726,Tahan
492,15990,2019-12-16,1605.0,1610.0,1565.0,1484.834351,16279800.0,0.0,2.977512,50.112717,80.762455,12.14,14.500617,Tahan
493,15994,2019-12-17,1585.0,1635.0,1555.0,1541.399414,46622200.0,1.0,7.853500,55.676841,134.244489,12.29,9.745929,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,MEDC.JK
0,13139,2018-01-02,753.439148,770.370361,753.439148,713.797363,45424968.0,1.0,15.726914,60.442577,79.238661,9.77,4.977856,Beli 100.0 saham
1,13143,2018-01-03,757.671936,770.370361,753.439148,709.809631,46648035.0,2.0,14.917566,59.654790,69.101356,9.15,6.195140,Beli 100.0 saham
2,13147,2018-01-04,770.370361,770.370361,757.671936,713.797363,49596080.0,3.0,14.431571,60.191531,74.229441,9.22,7.896003,Beli 100.0 saham
3,13151,2018-01-05,766.137573,812.698425,761.904785,757.661987,118722003.0,4.0,17.385515,65.425597,175.835426,9.22,2.734137,Beli 100.0 saham
4,13163,2018-01-08,829.629639,838.095215,804.232788,769.625061,97001533.0,0.0,20.456053,66.662260,250.291888,9.52,5.385585,Beli 100.0 saham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15975,2019-12-12,698.412720,715.343933,681.481506,661.957336,87890315.0,3.0,28.039621,66.014572,205.825025,13.94,0.220926,Tahan
491,15979,2019-12-13,715.343933,744.973572,711.111084,693.858887,113998066.0,4.0,33.547092,69.154002,219.392372,12.63,4.513726,Tahan
492,15991,2019-12-16,740.740723,804.232788,736.507935,757.661987,225414866.0,0.0,42.569465,74.103417,248.805834,12.14,14.500617,Tahan
493,15995,2019-12-17,800.000000,833.862427,757.671936,721.772766,169035575.0,1.0,46.290195,67.775334,217.871857,12.29,9.745929,Tahan


<a id='6.4.4'></a>
###7.5.4. Agen 4: SAC

In [84]:
#drop some columns
df_actions_and_account_value_sac = df_actions_and_account_value_sac.drop(['date', 'total_shares', 'account_value'], axis=1)

# create empty list
dataframes_list = []

#read the dataset of each stock, then put them on dataframes_list
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  mask = (df_temp['date'] >= TRADE_START_DATE) & (df_temp['date'] <= TRADE_END_DATE)
  df_temp = df_temp.loc[mask]
  dataframes_list.append(df_temp)
  #display(df_temp)

#separate each column of trading dataframe action, then put them on action_per_tic_list
action_per_tic_list = []
for i in df_actions_and_account_value_sac:
    df_temp = df_actions_and_account_value_sac[[i]]
    action_per_tic_list.append(df_temp)
    #display(df_temp)

# merging each dataframes_list members and each action_per_tic_list members
j=0
for i in dataframes_list:
  if j<numberOfTic:
   i['action'] = action_per_tic_list[j].iloc[:, 0].values.tolist()
   display(i)
   i.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActions/Fraksi3/4_SAC/df_states_and_actions_sac_'+list_ticker[j]+'.csv',index=False)
   j+=1

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13136,2018-01-02,AKRA.JK,1275.0,1285.0,1235.0,1008.418701,25448000.0,1.0,-12.741437,42.797321,-27.402704,9.77,4.977856,100.0
13140,2018-01-03,AKRA.JK,1260.0,1265.0,1230.0,1004.352600,34013000.0,2.0,-13.495459,42.414932,-48.046638,9.15,6.195140,100.0
13144,2018-01-04,AKRA.JK,1240.0,1250.0,1240.0,1008.418701,21831000.0,3.0,-13.608060,42.942315,-44.180827,9.22,7.896003,100.0
13148,2018-01-05,AKRA.JK,1240.0,1240.0,1230.0,1008.418701,22779000.0,4.0,-13.541203,42.942315,-60.188004,9.22,2.734137,0.0
13160,2018-01-08,AKRA.JK,1250.0,1290.0,1245.0,1049.080688,44097500.0,0.0,-10.090815,48.035401,71.309493,9.52,5.385585,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,AKRA.JK,754.0,756.0,740.0,672.013977,36956500.0,3.0,-1.634581,49.494913,40.825594,13.94,0.220926,0.0
15976,2019-12-13,AKRA.JK,758.0,770.0,754.0,684.491699,58397500.0,4.0,0.871302,51.475430,75.206993,12.63,4.513726,0.0
15988,2019-12-16,AKRA.JK,768.0,778.0,756.0,689.839233,62960000.0,0.0,3.251256,52.304633,89.005454,12.14,14.500617,0.0
15992,2019-12-17,AKRA.JK,774.0,778.0,766.0,689.839233,53565500.0,1.0,5.078840,52.304633,97.518591,12.29,9.745929,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13137,2018-01-02,BRPT.JK,454.0,454.0,440.0,438.999084,110809000.0,1.0,1.533834,53.320545,-60.693822,9.77,4.977856,100.0
13141,2018-01-03,BRPT.JK,450.0,450.0,424.0,417.536926,133037000.0,2.0,-0.774940,47.187261,-138.640370,9.15,6.195140,100.0
13145,2018-01-04,BRPT.JK,428.0,430.0,406.0,399.976959,206900500.0,3.0,-3.975773,43.000787,-226.780331,9.22,7.896003,100.0
13149,2018-01-05,BRPT.JK,410.0,422.0,410.0,409.732513,156436000.0,4.0,-5.660019,45.766110,-203.154314,9.22,2.734137,1.0
13161,2018-01-08,BRPT.JK,420.0,426.0,412.0,407.781372,165324500.0,0.0,-7.070730,45.311286,-184.295229,9.52,5.385585,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,BRPT.JK,1430.0,1435.0,1405.0,1400.250610,58912700.0,3.0,95.708893,68.859335,92.321350,13.94,0.220926,0.0
15977,2019-12-13,BRPT.JK,1415.0,1420.0,1380.0,1410.181396,75925900.0,4.0,93.046575,69.276538,83.889991,12.63,4.513726,0.0
15989,2019-12-16,BRPT.JK,1410.0,1430.0,1405.0,1395.285156,74485300.0,0.0,88.712049,67.865673,81.687791,12.14,14.500617,0.0
15993,2019-12-17,BRPT.JK,1405.0,1435.0,1380.0,1425.077637,72110900.0,1.0,86.681698,69.164937,78.838577,12.29,9.745929,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13138,2018-01-02,KLBF.JK,1690.0,1730.0,1675.0,1568.763916,17137900.0,1.0,13.548843,58.402810,189.233758,9.77,4.977856,100.0
13142,2018-01-03,KLBF.JK,1730.0,1730.0,1675.0,1537.025879,15571500.0,2.0,13.566010,53.885335,138.052325,9.15,6.195140,100.0
13146,2018-01-04,KLBF.JK,1695.0,1745.0,1685.0,1582.365845,17727400.0,3.0,17.041725,58.615967,190.347106,9.22,7.896003,100.0
13150,2018-01-05,KLBF.JK,1730.0,1760.0,1730.0,1595.967896,16178100.0,4.0,20.655719,59.892843,227.496530,9.22,2.734137,63.0
13162,2018-01-08,KLBF.JK,1770.0,1785.0,1740.0,1618.637939,12831400.0,0.0,25.060240,61.918655,237.953502,9.52,5.385585,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,KLBF.JK,1550.0,1590.0,1545.0,1480.120605,26446200.0,3.0,-0.613375,49.672440,42.118931,13.94,0.220926,0.0
15978,2019-12-13,KLBF.JK,1600.0,1600.0,1575.0,1508.403076,18588700.0,4.0,2.470021,52.781494,100.013048,12.63,4.513726,0.0
15990,2019-12-16,KLBF.JK,1605.0,1610.0,1565.0,1484.834351,16279800.0,0.0,2.977512,50.112717,80.762455,12.14,14.500617,0.0
15994,2019-12-17,KLBF.JK,1585.0,1635.0,1555.0,1541.399414,46622200.0,1.0,7.853500,55.676841,134.244489,12.29,9.745929,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13139,2018-01-02,MEDC.JK,753.439148,770.370361,753.439148,713.797363,45424968.0,1.0,15.726914,60.442577,79.238661,9.77,4.977856,0.0
13143,2018-01-03,MEDC.JK,757.671936,770.370361,753.439148,709.809631,46648035.0,2.0,14.917566,59.654790,69.101356,9.15,6.195140,0.0
13147,2018-01-04,MEDC.JK,770.370361,770.370361,757.671936,713.797363,49596080.0,3.0,14.431571,60.191531,74.229441,9.22,7.896003,0.0
13151,2018-01-05,MEDC.JK,766.137573,812.698425,761.904785,757.661987,118722003.0,4.0,17.385515,65.425597,175.835426,9.22,2.734137,0.0
13163,2018-01-08,MEDC.JK,829.629639,838.095215,804.232788,769.625061,97001533.0,0.0,20.456053,66.662260,250.291888,9.52,5.385585,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,MEDC.JK,698.412720,715.343933,681.481506,661.957336,87890315.0,3.0,28.039621,66.014572,205.825025,13.94,0.220926,0.0
15979,2019-12-13,MEDC.JK,715.343933,744.973572,711.111084,693.858887,113998066.0,4.0,33.547092,69.154002,219.392372,12.63,4.513726,0.0
15991,2019-12-16,MEDC.JK,740.740723,804.232788,736.507935,757.661987,225414866.0,0.0,42.569465,74.103417,248.805834,12.14,14.500617,0.0
15995,2019-12-17,MEDC.JK,800.000000,833.862427,757.671936,721.772766,169035575.0,1.0,46.290195,67.775334,217.871857,12.29,9.745929,0.0


####String Action Version

In [85]:
numberOfColumns = len(dataframes_list[0].columns)
numberOfRows = len(dataframes_list[0])

list_actionCell_str=[]

j=0
for j in range(len(i)):
  for i in dataframes_list:
    actionCell= i["action"].values[j]
    actionCell_str = actionCell.astype(str)
    if actionCell > 0:
      actionCell_str = "Beli " + actionCell_str + " saham"
    elif val == 0:
      actionCell_str = "Tahan"
    else:
      actionCell_str = "Jual " + actionCell_str + " saham"
    list_actionCell_str.append(actionCell_str)
  j+=1

arr_actions_str = np.array(list_actionCell_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

j=0
for i in dataframes_list:
  #drop a column
  i = i.drop(['action','tic'], axis=1)

  #reset index
  i=i.reset_index()

  #selecting column of actions in every tics  
  action_columns_str = df_actions_str.iloc[:,j]
  
  df_action_columns_str = pd.concat([i, action_columns_str], axis=1)
  df_action_columns_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActionsString/Fraksi3/4_SAC/df_states_and_actions_str_sac_'+list_ticker[j]+'.csv',index=False)
  display(df_action_columns_str)
  j+=1

,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,AKRA.JK
0,13136,2018-01-02,1275.0,1285.0,1235.0,1008.418701,25448000.0,1.0,-12.741437,42.797321,-27.402704,9.77,4.977856,Beli 100.0 saham
1,13140,2018-01-03,1260.0,1265.0,1230.0,1004.352600,34013000.0,2.0,-13.495459,42.414932,-48.046638,9.15,6.195140,Beli 100.0 saham
2,13144,2018-01-04,1240.0,1250.0,1240.0,1008.418701,21831000.0,3.0,-13.608060,42.942315,-44.180827,9.22,7.896003,Beli 100.0 saham
3,13148,2018-01-05,1240.0,1240.0,1230.0,1008.418701,22779000.0,4.0,-13.541203,42.942315,-60.188004,9.22,2.734137,Tahan
4,13160,2018-01-08,1250.0,1290.0,1245.0,1049.080688,44097500.0,0.0,-10.090815,48.035401,71.309493,9.52,5.385585,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15972,2019-12-12,754.0,756.0,740.0,672.013977,36956500.0,3.0,-1.634581,49.494913,40.825594,13.94,0.220926,Tahan
491,15976,2019-12-13,758.0,770.0,754.0,684.491699,58397500.0,4.0,0.871302,51.475430,75.206993,12.63,4.513726,Tahan
492,15988,2019-12-16,768.0,778.0,756.0,689.839233,62960000.0,0.0,3.251256,52.304633,89.005454,12.14,14.500617,Tahan
493,15992,2019-12-17,774.0,778.0,766.0,689.839233,53565500.0,1.0,5.078840,52.304633,97.518591,12.29,9.745929,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,BRPT.JK
0,13137,2018-01-02,454.0,454.0,440.0,438.999084,110809000.0,1.0,1.533834,53.320545,-60.693822,9.77,4.977856,Beli 100.0 saham
1,13141,2018-01-03,450.0,450.0,424.0,417.536926,133037000.0,2.0,-0.774940,47.187261,-138.640370,9.15,6.195140,Beli 100.0 saham
2,13145,2018-01-04,428.0,430.0,406.0,399.976959,206900500.0,3.0,-3.975773,43.000787,-226.780331,9.22,7.896003,Beli 100.0 saham
3,13149,2018-01-05,410.0,422.0,410.0,409.732513,156436000.0,4.0,-5.660019,45.766110,-203.154314,9.22,2.734137,Beli 1.0 saham
4,13161,2018-01-08,420.0,426.0,412.0,407.781372,165324500.0,0.0,-7.070730,45.311286,-184.295229,9.52,5.385585,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15973,2019-12-12,1430.0,1435.0,1405.0,1400.250610,58912700.0,3.0,95.708893,68.859335,92.321350,13.94,0.220926,Tahan
491,15977,2019-12-13,1415.0,1420.0,1380.0,1410.181396,75925900.0,4.0,93.046575,69.276538,83.889991,12.63,4.513726,Tahan
492,15989,2019-12-16,1410.0,1430.0,1405.0,1395.285156,74485300.0,0.0,88.712049,67.865673,81.687791,12.14,14.500617,Tahan
493,15993,2019-12-17,1405.0,1435.0,1380.0,1425.077637,72110900.0,1.0,86.681698,69.164937,78.838577,12.29,9.745929,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,KLBF.JK
0,13138,2018-01-02,1690.0,1730.0,1675.0,1568.763916,17137900.0,1.0,13.548843,58.402810,189.233758,9.77,4.977856,Beli 100.0 saham
1,13142,2018-01-03,1730.0,1730.0,1675.0,1537.025879,15571500.0,2.0,13.566010,53.885335,138.052325,9.15,6.195140,Beli 100.0 saham
2,13146,2018-01-04,1695.0,1745.0,1685.0,1582.365845,17727400.0,3.0,17.041725,58.615967,190.347106,9.22,7.896003,Beli 100.0 saham
3,13150,2018-01-05,1730.0,1760.0,1730.0,1595.967896,16178100.0,4.0,20.655719,59.892843,227.496530,9.22,2.734137,Beli 63.0 saham
4,13162,2018-01-08,1770.0,1785.0,1740.0,1618.637939,12831400.0,0.0,25.060240,61.918655,237.953502,9.52,5.385585,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15974,2019-12-12,1550.0,1590.0,1545.0,1480.120605,26446200.0,3.0,-0.613375,49.672440,42.118931,13.94,0.220926,Tahan
491,15978,2019-12-13,1600.0,1600.0,1575.0,1508.403076,18588700.0,4.0,2.470021,52.781494,100.013048,12.63,4.513726,Tahan
492,15990,2019-12-16,1605.0,1610.0,1565.0,1484.834351,16279800.0,0.0,2.977512,50.112717,80.762455,12.14,14.500617,Tahan
493,15994,2019-12-17,1585.0,1635.0,1555.0,1541.399414,46622200.0,1.0,7.853500,55.676841,134.244489,12.29,9.745929,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,MEDC.JK
0,13139,2018-01-02,753.439148,770.370361,753.439148,713.797363,45424968.0,1.0,15.726914,60.442577,79.238661,9.77,4.977856,Tahan
1,13143,2018-01-03,757.671936,770.370361,753.439148,709.809631,46648035.0,2.0,14.917566,59.654790,69.101356,9.15,6.195140,Tahan
2,13147,2018-01-04,770.370361,770.370361,757.671936,713.797363,49596080.0,3.0,14.431571,60.191531,74.229441,9.22,7.896003,Tahan
3,13151,2018-01-05,766.137573,812.698425,761.904785,757.661987,118722003.0,4.0,17.385515,65.425597,175.835426,9.22,2.734137,Tahan
4,13163,2018-01-08,829.629639,838.095215,804.232788,769.625061,97001533.0,0.0,20.456053,66.662260,250.291888,9.52,5.385585,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15975,2019-12-12,698.412720,715.343933,681.481506,661.957336,87890315.0,3.0,28.039621,66.014572,205.825025,13.94,0.220926,Tahan
491,15979,2019-12-13,715.343933,744.973572,711.111084,693.858887,113998066.0,4.0,33.547092,69.154002,219.392372,12.63,4.513726,Tahan
492,15991,2019-12-16,740.740723,804.232788,736.507935,757.661987,225414866.0,0.0,42.569465,74.103417,248.805834,12.14,14.500617,Tahan
493,15995,2019-12-17,800.000000,833.862427,757.671936,721.772766,169035575.0,1.0,46.290195,67.775334,217.871857,12.29,9.745929,Tahan


<a id='6.5.5'></a>
###7.4.5. Agen 5: DDPG

In [86]:
#drop some columns
df_actions_and_account_value_ddpg = df_actions_and_account_value_ddpg.drop(['date', 'total_shares', 'account_value'], axis=1)

# create empty list
dataframes_list = []

#read the dataset of each stock, then put them on dataframes_list
for stock in processed_full['tic'].unique():
  df_temp = processed_full.loc[processed_full['tic'].isin([stock])]
  mask = (df_temp['date'] >= TRADE_START_DATE) & (df_temp['date'] <= TRADE_END_DATE)
  df_temp = df_temp.loc[mask]
  dataframes_list.append(df_temp)
  #display(df_temp)

#separate each column of trading dataframe action, then put them on action_per_tic_list
action_per_tic_list = []
for i in df_actions_and_account_value_ddpg:
    df_temp = df_actions_and_account_value_ddpg[[i]]
    action_per_tic_list.append(df_temp)
    #display(df_temp)

# merging each dataframes_list members and each action_per_tic_list members
j=0
for i in dataframes_list:
  if j<numberOfTic:
   i['action'] = action_per_tic_list[j].iloc[:, 0].values.tolist()
   display(i)
   i.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActions/Fraksi3/5_DDPG/df_states_and_actions_ddpg_'+list_ticker[j]+'.csv',index=False)
   j+=1

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13136,2018-01-02,AKRA.JK,1275.0,1285.0,1235.0,1008.418701,25448000.0,1.0,-12.741437,42.797321,-27.402704,9.77,4.977856,0.0
13140,2018-01-03,AKRA.JK,1260.0,1265.0,1230.0,1004.352600,34013000.0,2.0,-13.495459,42.414932,-48.046638,9.15,6.195140,0.0
13144,2018-01-04,AKRA.JK,1240.0,1250.0,1240.0,1008.418701,21831000.0,3.0,-13.608060,42.942315,-44.180827,9.22,7.896003,0.0
13148,2018-01-05,AKRA.JK,1240.0,1240.0,1230.0,1008.418701,22779000.0,4.0,-13.541203,42.942315,-60.188004,9.22,2.734137,0.0
13160,2018-01-08,AKRA.JK,1250.0,1290.0,1245.0,1049.080688,44097500.0,0.0,-10.090815,48.035401,71.309493,9.52,5.385585,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15972,2019-12-12,AKRA.JK,754.0,756.0,740.0,672.013977,36956500.0,3.0,-1.634581,49.494913,40.825594,13.94,0.220926,0.0
15976,2019-12-13,AKRA.JK,758.0,770.0,754.0,684.491699,58397500.0,4.0,0.871302,51.475430,75.206993,12.63,4.513726,0.0
15988,2019-12-16,AKRA.JK,768.0,778.0,756.0,689.839233,62960000.0,0.0,3.251256,52.304633,89.005454,12.14,14.500617,0.0
15992,2019-12-17,AKRA.JK,774.0,778.0,766.0,689.839233,53565500.0,1.0,5.078840,52.304633,97.518591,12.29,9.745929,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13137,2018-01-02,BRPT.JK,454.0,454.0,440.0,438.999084,110809000.0,1.0,1.533834,53.320545,-60.693822,9.77,4.977856,100.0
13141,2018-01-03,BRPT.JK,450.0,450.0,424.0,417.536926,133037000.0,2.0,-0.774940,47.187261,-138.640370,9.15,6.195140,100.0
13145,2018-01-04,BRPT.JK,428.0,430.0,406.0,399.976959,206900500.0,3.0,-3.975773,43.000787,-226.780331,9.22,7.896003,100.0
13149,2018-01-05,BRPT.JK,410.0,422.0,410.0,409.732513,156436000.0,4.0,-5.660019,45.766110,-203.154314,9.22,2.734137,100.0
13161,2018-01-08,BRPT.JK,420.0,426.0,412.0,407.781372,165324500.0,0.0,-7.070730,45.311286,-184.295229,9.52,5.385585,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15973,2019-12-12,BRPT.JK,1430.0,1435.0,1405.0,1400.250610,58912700.0,3.0,95.708893,68.859335,92.321350,13.94,0.220926,0.0
15977,2019-12-13,BRPT.JK,1415.0,1420.0,1380.0,1410.181396,75925900.0,4.0,93.046575,69.276538,83.889991,12.63,4.513726,0.0
15989,2019-12-16,BRPT.JK,1410.0,1430.0,1405.0,1395.285156,74485300.0,0.0,88.712049,67.865673,81.687791,12.14,14.500617,0.0
15993,2019-12-17,BRPT.JK,1405.0,1435.0,1380.0,1425.077637,72110900.0,1.0,86.681698,69.164937,78.838577,12.29,9.745929,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13138,2018-01-02,KLBF.JK,1690.0,1730.0,1675.0,1568.763916,17137900.0,1.0,13.548843,58.402810,189.233758,9.77,4.977856,0.0
13142,2018-01-03,KLBF.JK,1730.0,1730.0,1675.0,1537.025879,15571500.0,2.0,13.566010,53.885335,138.052325,9.15,6.195140,0.0
13146,2018-01-04,KLBF.JK,1695.0,1745.0,1685.0,1582.365845,17727400.0,3.0,17.041725,58.615967,190.347106,9.22,7.896003,0.0
13150,2018-01-05,KLBF.JK,1730.0,1760.0,1730.0,1595.967896,16178100.0,4.0,20.655719,59.892843,227.496530,9.22,2.734137,0.0
13162,2018-01-08,KLBF.JK,1770.0,1785.0,1740.0,1618.637939,12831400.0,0.0,25.060240,61.918655,237.953502,9.52,5.385585,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15974,2019-12-12,KLBF.JK,1550.0,1590.0,1545.0,1480.120605,26446200.0,3.0,-0.613375,49.672440,42.118931,13.94,0.220926,0.0
15978,2019-12-13,KLBF.JK,1600.0,1600.0,1575.0,1508.403076,18588700.0,4.0,2.470021,52.781494,100.013048,12.63,4.513726,0.0
15990,2019-12-16,KLBF.JK,1605.0,1610.0,1565.0,1484.834351,16279800.0,0.0,2.977512,50.112717,80.762455,12.14,14.500617,0.0
15994,2019-12-17,KLBF.JK,1585.0,1635.0,1555.0,1541.399414,46622200.0,1.0,7.853500,55.676841,134.244489,12.29,9.745929,0.0


,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,action
13139,2018-01-02,MEDC.JK,753.439148,770.370361,753.439148,713.797363,45424968.0,1.0,15.726914,60.442577,79.238661,9.77,4.977856,100.0
13143,2018-01-03,MEDC.JK,757.671936,770.370361,753.439148,709.809631,46648035.0,2.0,14.917566,59.654790,69.101356,9.15,6.195140,100.0
13147,2018-01-04,MEDC.JK,770.370361,770.370361,757.671936,713.797363,49596080.0,3.0,14.431571,60.191531,74.229441,9.22,7.896003,100.0
13151,2018-01-05,MEDC.JK,766.137573,812.698425,761.904785,757.661987,118722003.0,4.0,17.385515,65.425597,175.835426,9.22,2.734137,100.0
13163,2018-01-08,MEDC.JK,829.629639,838.095215,804.232788,769.625061,97001533.0,0.0,20.456053,66.662260,250.291888,9.52,5.385585,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15975,2019-12-12,MEDC.JK,698.412720,715.343933,681.481506,661.957336,87890315.0,3.0,28.039621,66.014572,205.825025,13.94,0.220926,0.0
15979,2019-12-13,MEDC.JK,715.343933,744.973572,711.111084,693.858887,113998066.0,4.0,33.547092,69.154002,219.392372,12.63,4.513726,0.0
15991,2019-12-16,MEDC.JK,740.740723,804.232788,736.507935,757.661987,225414866.0,0.0,42.569465,74.103417,248.805834,12.14,14.500617,0.0
15995,2019-12-17,MEDC.JK,800.000000,833.862427,757.671936,721.772766,169035575.0,1.0,46.290195,67.775334,217.871857,12.29,9.745929,0.0


####String Action Version

In [87]:
numberOfColumns = len(dataframes_list[0].columns)
numberOfRows = len(dataframes_list[0])

list_actionCell_str=[]

j=0
for j in range(len(i)):
  for i in dataframes_list:
    actionCell= i["action"].values[j]
    actionCell_str = actionCell.astype(str)
    if actionCell > 0:
      actionCell_str = "Beli " + actionCell_str + " saham"
    elif val == 0:
      actionCell_str = "Tahan"
    else:
      actionCell_str = "Jual " + actionCell_str + " saham"
    list_actionCell_str.append(actionCell_str)
  j+=1

arr_actions_str = np.array(list_actionCell_str)
arr_actions_str = np.array_split(arr_actions_str, numberOfRows)

df_actions_str = pd.DataFrame(arr_actions_str, columns = list_ticker)

j=0
for i in dataframes_list:
  #drop a column
  i = i.drop(['action','tic'], axis=1)

  #reset index
  i=i.reset_index()

  #selecting column of actions in every tics  
  action_columns_str = df_actions_str.iloc[:,j]
  
  df_action_columns_str = pd.concat([i, action_columns_str], axis=1)
  df_action_columns_str.to_csv('/content/drive/MyDrive/Fraksi_FullVersion_DRLforMultipleStockTradingUsingFinRL/StatesAndActionsString/Fraksi3/5_DDPG/df_states_and_actions_str_ddpg_'+list_ticker[j]+'.csv',index=False)
  display(df_action_columns_str)
  j+=1

,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,AKRA.JK
0,13136,2018-01-02,1275.0,1285.0,1235.0,1008.418701,25448000.0,1.0,-12.741437,42.797321,-27.402704,9.77,4.977856,Tahan
1,13140,2018-01-03,1260.0,1265.0,1230.0,1004.352600,34013000.0,2.0,-13.495459,42.414932,-48.046638,9.15,6.195140,Tahan
2,13144,2018-01-04,1240.0,1250.0,1240.0,1008.418701,21831000.0,3.0,-13.608060,42.942315,-44.180827,9.22,7.896003,Tahan
3,13148,2018-01-05,1240.0,1240.0,1230.0,1008.418701,22779000.0,4.0,-13.541203,42.942315,-60.188004,9.22,2.734137,Tahan
4,13160,2018-01-08,1250.0,1290.0,1245.0,1049.080688,44097500.0,0.0,-10.090815,48.035401,71.309493,9.52,5.385585,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15972,2019-12-12,754.0,756.0,740.0,672.013977,36956500.0,3.0,-1.634581,49.494913,40.825594,13.94,0.220926,Tahan
491,15976,2019-12-13,758.0,770.0,754.0,684.491699,58397500.0,4.0,0.871302,51.475430,75.206993,12.63,4.513726,Tahan
492,15988,2019-12-16,768.0,778.0,756.0,689.839233,62960000.0,0.0,3.251256,52.304633,89.005454,12.14,14.500617,Tahan
493,15992,2019-12-17,774.0,778.0,766.0,689.839233,53565500.0,1.0,5.078840,52.304633,97.518591,12.29,9.745929,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,BRPT.JK
0,13137,2018-01-02,454.0,454.0,440.0,438.999084,110809000.0,1.0,1.533834,53.320545,-60.693822,9.77,4.977856,Beli 100.0 saham
1,13141,2018-01-03,450.0,450.0,424.0,417.536926,133037000.0,2.0,-0.774940,47.187261,-138.640370,9.15,6.195140,Beli 100.0 saham
2,13145,2018-01-04,428.0,430.0,406.0,399.976959,206900500.0,3.0,-3.975773,43.000787,-226.780331,9.22,7.896003,Beli 100.0 saham
3,13149,2018-01-05,410.0,422.0,410.0,409.732513,156436000.0,4.0,-5.660019,45.766110,-203.154314,9.22,2.734137,Beli 100.0 saham
4,13161,2018-01-08,420.0,426.0,412.0,407.781372,165324500.0,0.0,-7.070730,45.311286,-184.295229,9.52,5.385585,Beli 100.0 saham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15973,2019-12-12,1430.0,1435.0,1405.0,1400.250610,58912700.0,3.0,95.708893,68.859335,92.321350,13.94,0.220926,Tahan
491,15977,2019-12-13,1415.0,1420.0,1380.0,1410.181396,75925900.0,4.0,93.046575,69.276538,83.889991,12.63,4.513726,Tahan
492,15989,2019-12-16,1410.0,1430.0,1405.0,1395.285156,74485300.0,0.0,88.712049,67.865673,81.687791,12.14,14.500617,Tahan
493,15993,2019-12-17,1405.0,1435.0,1380.0,1425.077637,72110900.0,1.0,86.681698,69.164937,78.838577,12.29,9.745929,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,KLBF.JK
0,13138,2018-01-02,1690.0,1730.0,1675.0,1568.763916,17137900.0,1.0,13.548843,58.402810,189.233758,9.77,4.977856,Tahan
1,13142,2018-01-03,1730.0,1730.0,1675.0,1537.025879,15571500.0,2.0,13.566010,53.885335,138.052325,9.15,6.195140,Tahan
2,13146,2018-01-04,1695.0,1745.0,1685.0,1582.365845,17727400.0,3.0,17.041725,58.615967,190.347106,9.22,7.896003,Tahan
3,13150,2018-01-05,1730.0,1760.0,1730.0,1595.967896,16178100.0,4.0,20.655719,59.892843,227.496530,9.22,2.734137,Tahan
4,13162,2018-01-08,1770.0,1785.0,1740.0,1618.637939,12831400.0,0.0,25.060240,61.918655,237.953502,9.52,5.385585,Tahan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15974,2019-12-12,1550.0,1590.0,1545.0,1480.120605,26446200.0,3.0,-0.613375,49.672440,42.118931,13.94,0.220926,Tahan
491,15978,2019-12-13,1600.0,1600.0,1575.0,1508.403076,18588700.0,4.0,2.470021,52.781494,100.013048,12.63,4.513726,Tahan
492,15990,2019-12-16,1605.0,1610.0,1565.0,1484.834351,16279800.0,0.0,2.977512,50.112717,80.762455,12.14,14.500617,Tahan
493,15994,2019-12-17,1585.0,1635.0,1555.0,1541.399414,46622200.0,1.0,7.853500,55.676841,134.244489,12.29,9.745929,Tahan


,index,date,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence,MEDC.JK
0,13139,2018-01-02,753.439148,770.370361,753.439148,713.797363,45424968.0,1.0,15.726914,60.442577,79.238661,9.77,4.977856,Beli 100.0 saham
1,13143,2018-01-03,757.671936,770.370361,753.439148,709.809631,46648035.0,2.0,14.917566,59.654790,69.101356,9.15,6.195140,Beli 100.0 saham
2,13147,2018-01-04,770.370361,770.370361,757.671936,713.797363,49596080.0,3.0,14.431571,60.191531,74.229441,9.22,7.896003,Beli 100.0 saham
3,13151,2018-01-05,766.137573,812.698425,761.904785,757.661987,118722003.0,4.0,17.385515,65.425597,175.835426,9.22,2.734137,Beli 100.0 saham
4,13163,2018-01-08,829.629639,838.095215,804.232788,769.625061,97001533.0,0.0,20.456053,66.662260,250.291888,9.52,5.385585,Beli 100.0 saham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,15975,2019-12-12,698.412720,715.343933,681.481506,661.957336,87890315.0,3.0,28.039621,66.014572,205.825025,13.94,0.220926,Tahan
491,15979,2019-12-13,715.343933,744.973572,711.111084,693.858887,113998066.0,4.0,33.547092,69.154002,219.392372,12.63,4.513726,Tahan
492,15991,2019-12-16,740.740723,804.232788,736.507935,757.661987,225414866.0,0.0,42.569465,74.103417,248.805834,12.14,14.500617,Tahan
493,15995,2019-12-17,800.000000,833.862427,757.671936,721.772766,169035575.0,1.0,46.290195,67.775334,217.871857,12.29,9.745929,Tahan


<a id='7'></a>
# Bagian 8: Performa Backtesting
Backtesting memiliki peran kunci dalam mengevaluasi kinerja strategi perdagangan. Alat backtesting otomatis lebih disukai karena dapat meminimalisir kesalahan manusia. Backtesting dapat dilakukan dengan menggunakan paket Quantopian pyfolio untuk menguji strategi perdagangan pada notebook ini. Backtesting tersebut mudah digunakan dan terdiri dari berbagai plot yang memberikan gambaran komprehensif tentang kinerja strategi perdagangan.

<a id='7.1'></a>
## 8.1 Status BackTesting


###8.1.1 Agen A2C

In [88]:
print("===========Get Backtest Results of A2C Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_a2c)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_a2c_"+now+'.csv')

===========Get Backtest Results of A2C Model===========
Annual return         -0.166535
Cumulative returns    -0.300802
Annual volatility      0.329278
Sharpe ratio          -0.389866
Calmar ratio          -0.358319
Stability              0.388388
Max drawdown          -0.464767
Omega ratio            0.932470
Sortino ratio         -0.557623
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.121512
Daily value at risk   -0.041995
dtype: float64


###8.1.2 Agen ppo

In [89]:
print("===========Get Backtest Results of PPO Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_ppo)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_ppo_"+now+'.csv')

===========Get Backtest Results of PPO Model===========
Annual return          0.018484
Cumulative returns     0.036630
Annual volatility      0.311232
Sharpe ratio           0.213902
Calmar ratio           0.060562
Stability              0.187752
Max drawdown          -0.305199
Omega ratio            1.040472
Sortino ratio          0.312947
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.992278
Daily value at risk   -0.038947
dtype: float64


###8.1.3 Agen TD3

In [90]:
print("===========Get Backtest Results of TD3 Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_td3)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_td3_"+now+'.csv')

===========Get Backtest Results of TD3 Model===========
Annual return          0.341577
Cumulative returns     0.781040
Annual volatility      0.391348
Sharpe ratio           0.945234
Calmar ratio           0.686491
Stability              0.069653
Max drawdown          -0.497570
Omega ratio            1.186111
Sortino ratio          1.588033
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.473150
Daily value at risk   -0.047837
dtype: float64


###8.1.4 Agen SAC

In [91]:
print("===========Get Backtest Results of SAC Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_sac)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_sac_"+now+'.csv')

===========Get Backtest Results of SAC Model===========
Annual return          0.101960
Cumulative returns     0.210113
Annual volatility      0.251939
Sharpe ratio           0.512110
Calmar ratio           0.296442
Stability              0.260967
Max drawdown          -0.343947
Omega ratio            1.092667
Sortino ratio          0.733465
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.040470
Daily value at risk   -0.031229
dtype: float64


###8.1.5 Agen DDPG

In [92]:
print("===========Get Backtest Results of DDPG Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_ddpg)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_ddpg_"+now+'.csv')

===========Get Backtest Results of DDPG Model===========
Annual return         -0.084579
Cumulative returns    -0.159355
Annual volatility      0.498044
Sharpe ratio           0.065931
Calmar ratio          -0.139522
Stability              0.682786
Max drawdown          -0.606206
Omega ratio            1.012159
Sortino ratio          0.107660
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.303314
Daily value at risk   -0.062617
dtype: float64
